## Import

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import chardet
import numpy as np
import scipy 
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothic'
import pandas as pd
from pyproj import Proj, transform

### Data Preprocessing

In [ ]:
DATA_PATH = './data'

# UTF-8 파일 읽기
def encoding_utf(DATA_PATH, filename):
    file_path = os.path.join(DATA_PATH, filename)
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    encoding = result['encoding']

    # 파일 읽기
    if filename.split('.')[-1] == 'csv':
        return pd.read_csv(file_path, encoding=encoding, sep='\t')
    if filename.split('.')[-1] == 'xlsx' or filename.split('.')[-1] == 'xls':
        return pd.read_excel(file_path)

def view_columns(clinic_dfs):
    for key, value in clinic_dfs.items():
        print(f"{key} col : {value.columns.values}") 

clinic_dfs = {
    'fig' : encoding_utf(DATA_PATH, 'clinic/pig.xlsx'),
    'clinic_rate' : encoding_utf(DATA_PATH, 'clinic/clinic_rate.xlsx'),
    'wadge' : encoding_utf(DATA_PATH, 'clinic/wadge.xlsx'),
}
view_columns(clinic_dfs)
clinic_infra_dfs = {
    'clinic' :  encoding_utf(DATA_PATH, 'clinic_infra/clinic.csv'),
    'dentist' :  encoding_utf(DATA_PATH, 'clinic_infra/dentist.xlsx'),
    'mom' :  encoding_utf(DATA_PATH, 'clinic_infra/mom.xlsx'),
    'pharmacy' :  encoding_utf(DATA_PATH, 'clinic_infra/pharmacy.xlsx'),
}
view_columns(clinic_infra_dfs)
infra_dfs = {
    'infra' :  encoding_utf(DATA_PATH, 'infra/infra.xls'),
}
view_columns(infra_dfs)
population_dfs = {
    'all' :  encoding_utf(DATA_PATH, 'population/all.xlsx'), # 전체 인구
    'basic' :  encoding_utf(DATA_PATH, 'population/basic.xlsx'), #기초수급자
    'child' :  encoding_utf(DATA_PATH, 'population/child.xlsx'), #어린이
    'company' :  encoding_utf(DATA_PATH, 'population/human.xlsx'), #모든 인간 건강보험료
}
view_columns(population_dfs)
    

### 세부 칼럼 데이터 전처리

### clinic 관련 정보

#### fig : 비만환자 지역별 일일환자 수 및 연 입원일수, 월 요양급여 비용

In [ ]:
clinic_dfs['fig'].to_csv('./pre_data/clinic/fig.csv')

In [ ]:
################### clinic/fig :비만환자 평균 전처리 ###########################
# '지역'을 기준으로 그룹화하여 각 그룹의 '환자수', '입내원일수', '요양급여비용' 합계 계산
clinic_dfs['fig'] = clinic_dfs['fig'].groupby('지역').agg({
    '환자수': 'mean',
    '입내원일수': 'mean',
    '요양급여비용': 'mean'
}).reset_index()
# 열 이름 변경
clinic_dfs['fig'] = clinic_dfs['fig'].rename(columns={'환자수': '일일환자수', '요양급여비용': '월 요양급여비용','입내원일수': '연 입원일수', })
# 그래프 플롯
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('지역')
ax1.set_ylabel('일일환자수', color=color)
ax1.bar(clinic_dfs['fig']['지역'], clinic_dfs['fig']['일일환자수'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('연 입원일수', color=color)
ax2.plot(clinic_dfs['fig']['지역'], clinic_dfs['fig']['연 입원일수'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

ax3 = ax1.twinx()
color = 'tab:green'
ax3.spines['right'].set_position(('outward', 60))
ax3.set_ylabel('월 요양급여비용', color=color)
ax3.plot(clinic_dfs['fig']['지역'], clinic_dfs['fig']['월 요양급여비용'], color=color)
ax3.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.title('지역별 일일환자수, 연 입원일수, 월 요양급여비용')
plt.show()

#### clinic rate : 지역별 내원일수, 진료일수, 총진료비 추이 

In [ ]:
################### clinic/clinic_rate :비만환자 평균 전처리 ###########################
df_clinic_rate = clinic_dfs['clinic_rate']
print(df_clinic_rate.shape[0])
# ',' 제거 및 숫자로 변환
numeric_columns = ['지급건수', '내원일수', '진료일수', '총진료비', '공단부담', '본인부담']
for col in numeric_columns:
    df_clinic_rate[col] = pd.to_numeric(df_clinic_rate[col].str.replace(',', ''), errors='coerce')

clinic_dfs['clinic_rate'] = df_clinic_rate[['지역','연도','지급건수','내원일수', '진료일수', '총진료비']]
# 데이터프레임을 연도, 지역으로 그룹화하여 각 항목의 합계 계산
grouped_df = df_clinic_rate.groupby(['연도', '지역']).agg({
    '내원일수': 'sum',
    '진료일수': 'sum',
    '총진료비': 'sum'
}).reset_index()

# 시각화
sns.lineplot(data=grouped_df, x='연도', y='내원일수', hue='지역', palette=['red', 'green', 'blue'])
plt.title('내원일수 추이')
plt.show()

sns.lineplot(data=grouped_df, x='연도', y='진료일수', hue='지역', palette=['red', 'green', 'blue'])
plt.title('진료일수 추이')
plt.show()

sns.lineplot(data=grouped_df, x='연도', y='총진료비', hue='지역', palette=['red', 'green', 'blue'])
plt.title('총진료비 추이')
plt.show()

#### wadge : 건강보험료 금액 : 시군구	: 연도 : 건수 합계 : 금액 합계 : 직장 건수 : 직장 금액 : 공교 건수 : 공교 금액 : 지역 건수 : 지역 금액

In [ ]:
# clinic_dfs['wadge'] = clinic_dfs['wadge'][['시군구', '연도', '건수 합계', '금액 합계']]
# clinic_dfs['wadge'] = clinic_dfs['wadge'].rename(columns={'건수 합계': '건강보험료 건수 합계', '금액 합계': '건강보험료 금액 합계'})

# clinic_dfs['wadge']
# 연도별로 그룹화하여 건강보험료 건수와 금액 합계의 평균 계산
print(clinic_dfs['wadge'].shape[0])
# Modify if your column names are different
grouped_df = clinic_dfs['wadge'].groupby(['연도', '시군구']).agg({
    '건수 합계': 'sum',
    '금액 합계': 'sum'
}).reset_index()

# Plot for '건강보험료 건수 합계'
plt.figure(figsize=(12, 8))
for region in clinic_dfs['wadge']['시군구'].unique():
    region_data = grouped_df[grouped_df['시군구'] == region]
    plt.plot(region_data['연도'], region_data['건수 합계'], label=f'{region} 건수')

plt.title('연도별 시군구에 따른 건강보험료 건수 합계')
plt.xlabel('연도')
plt.ylabel('건수 합계')
plt.legend()
plt.show()

# Plot for '건강보험료 금액 합계'
plt.figure(figsize=(12, 8))
for region in clinic_dfs['wadge']['시군구'].unique():
    region_data = grouped_df[grouped_df['시군구'] == region]
    plt.plot(region_data['연도'], region_data['금액 합계'], label=f'{region} 금액')

plt.title('연도별 시군구에 따른 건강보험료 금액 합계')
plt.xlabel('연도')
plt.ylabel('금액 합계')
plt.legend()
plt.show()

### clinic_infra : clinic, dentist, mom, pharmacy

#### clinic : 지역, 병원명, 의사수, 위도, 경도

In [ ]:
# 열 이름 변경
# clinic_infra_dfs['clinic'] = clinic_infra_dfs['clinic'].rename(columns={'요양기관명' : '병원명'}) # 1번만 시도
# clinic_infra_dfs['clinic']
# 기지역구별 병원수와 총의사수의 평균 계산
avg_hospital_data = clinic_infra_dfs['clinic'].groupby('지역').agg({'요양기관명': 'count', '총의사수': 'mean'}).reset_index()
avg_hospital_data = avg_hospital_data.rename(columns={'요양기관명': '병원수 평균', '총의사수': '의사수 평균'})
print(avg_hospital_data)

# 그래프 그리기
fig, ax1 = plt.subplots()

df = avg_hospital_data
# 병원수를 막대그래프로 그리기
ax1.bar(df['지역'], df['병원수 평균'], color='b', alpha=0.7, label='병원수 평균')
ax1.set_xlabel('지역')
ax1.set_ylabel('병원수 평균', color='b')
ax1.tick_params('y', colors='b')
ax1.legend(loc='upper left')

# 의사수를 선그래프로 그리기
ax2 = ax1.twinx()
ax2.plot(df['지역'], df['의사수 평균'], color='r', marker='o', label='의사수 평균')
ax2.set_ylabel('의사수 평균', color='r')
ax2.tick_params('y', colors='r')
ax2.legend(loc='upper right')

plt.title('지역별 병원수와 의사수 평균')
plt.show()
plt.close()

# 지역별로 그룹화하여 25%와 75% 사이의 값을 계산
grouped_data = clinic_infra_dfs['clinic'].groupby('지역').agg({
    '총의사수': lambda x: x[(x >= x.quantile(0.25)) & (x <= x.quantile(0.75))].mean(),
    '위도': 'mean',
    '경도': 'mean'
}).reset_index()

# 결과 출력
print(f"처인구는 특히 대형병원이 크게 존재하는 결측치 이상이 있기 떄문에 25~75% 평균값으로 확인")
print(grouped_data)
clinic_infra_dfs['clinic']

In [ ]:
!rm -rf ~/.cache/matplotlib

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothic'

# Joint Plot 그리기
plt.figure(figsize=(24, 8))
sns.jointplot(x='병원수 평균', y='의사수 평균', data=avg_hospital_data, kind='reg', height=10, scatter_kws={'s': 150, 'color': 'skyblue'}, line_kws={'color': 'red'})

# 각 지역에 대한 텍스트 추가
for i in range(len(avg_hospital_data)):
    plt.text(avg_hospital_data['병원수 평균'][i], avg_hospital_data['의사수 평균'][i], avg_hospital_data['지역'][i], fontsize=12)

plt.title('병원수 평균과 의사수 평균의 Joint Plot', fontsize=16)
plt.tight_layout()
plt.show()


#### dentist : 지역, 병원명, 업업상태명, 의사수, 위도, 경도, 폐업일자

In [ ]:
clinic_infra_dfs['dentist'] = clinic_infra_dfs['dentist'].rename(columns={'소재지지번주소' : '지역', '사업장명' : '병원명', '의료인수(명)':'의사수', 'WGS84위도' : '위도', 'WGS84경도' : '경도'}) # 1번만 시도
# '영업상태명'이 '폐업 등'인 경우를 '폐업'으로 처리
clinic_infra_dfs['dentist']['영업상태명'] = clinic_infra_dfs['dentist']['영업상태명'].apply(lambda x: '폐업' if '폐업' in x else x)
df = pd.DataFrame(clinic_infra_dfs['dentist'])

# '영업상태명'이 '영업/정상'인 row 선택
df_open = df[df['영업상태명'] == '영업/정상']

# 각 지역별로 병원수와 의사수 플로팅
plt.figure(figsize=(12, 6))

for region in df_open['지역'].unique():
    region_data = df_open[df_open['지역'] == region]
    plt.bar(region, region_data['병원명'].count(), label=f'{region} 병원수')

for region in df_open['지역'].unique():
    region_data = df_open[df_open['지역'] == region]
    plt.plot(region, region_data['의사수'].sum(), marker='o', label=f'{region} 의사수 합계')

plt.title('지역별 영업중인 치과 병원수 및 치과 의사수 합계')
plt.xlabel('지역')
plt.ylabel('병원수 및 의사수 합계')
plt.legend()
plt.show()

#### mom : 산후조리원 위도 경도 임산부정원수

In [ ]:
import pandas as pd
from pyproj import Proj, transform

# Assuming clinic_infra_dfs is a dictionary
mom_df = pd.DataFrame.from_dict(clinic_infra_dfs['mom'])

mom_df
# 좌표계 정의
original_crs = Proj(init='epsg:2097')  # UTM-K
target_crs = Proj(init='epsg:4326')    # WGS84 (위도, 경도)

# 좌표 변환 함수 정의
def convert_coordinates(x, y):
    lon, lat = transform(original_crs, target_crs, x, y)
    return lat, lon

# '위도'와 '경도' 열 업데이트
mom_df[['좌표정보(X)', '좌표정보(Y)']] = mom_df.apply(lambda row: pd.Series(convert_coordinates(row['좌표정보(X)'], row['좌표정보(Y)'])), axis=1)

# 칼럼 이름 변경
clinic_infra_dfs['mom'] = mom_df.rename(columns = {'좌표정보(X)' : '위도', '좌표정보(Y)' : '경도'})

# 변환된 데이터프레임 확인
clinic_infra_dfs['mom']

#보정
# google maps : 37.316245, 127.124620
x = 37.316095 - clinic_infra_dfs['mom']['위도'][1]  
y = 127.124214 - clinic_infra_dfs['mom']['경도'][1]
clinic_infra_dfs['mom']['위도'] += x 
clinic_infra_dfs['mom']['경도'] += y

In [ ]:


# 확인: 사랑돌봄산후조리원 37.244370, 127.068802 
x = 37.2444 - clinic_infra_dfs['mom']['위도'][3]
y = 127.0688 - clinic_infra_dfs['mom']['경도'][3] 
clinic_infra_dfs['mom']['위도'] += x 
clinic_infra_dfs['mom']['경도'] += y

In [ ]:
clinic_infra_dfs['mom']
clinic_infra_dfs['mom']['위도']

In [ ]:
clinic_infra_dfs['mom'].to_csv('./pre_data/clinic_infra/mom.csv')

#### pharmacy : 지역구별 약국 위도, 경도

In [ ]:
print(clinic_infra_dfs['pharmacy'].shape[0])

In [ ]:
clinic_infra_dfs['pharmacy'] = clinic_infra_dfs['pharmacy'].rename(columns = {'지역구':'지역', '좌표(x)' : '경도', '좌표(y)' : '위도'})
clinic_infra_dfs['pharmacy'][['위도', '경도']] = clinic_infra_dfs['pharmacy'][['경도','위도', ]]
clinic_infra_dfs['pharmacy'] = clinic_infra_dfs['pharmacy'].rename(columns = {'경도' : '위도', '위도' : '경도'})
clinic_infra_dfs['pharmacy']

### infra / emergency_clinic_location(응급 의료서비스 취약 주거지역 정보), walk_infra(도보접근가능시설수)

#### emergency_clinic_location(응급 의료서비스 취약 주거지역 정보)

In [ ]:
price_building = pd.read_csv('./data/infra/emergency_clinic_location.csv')
print(price_building.shape[0])
##  데이터 전처리
# 필요한 칼럼만 가지고 오기
price_building = price_building[['SIGNGU_CD', 'EMDL_CD', 'LNNO_ADRES', 'XCRDT', 'YCRDT', 'NRB_GNHSP_NM', 'NRB_GNHSP_ADRES', 'NRB_GNHSP_DSTNC', 'NRB_EGYT_NM', 'NRB_EGYT_DSTNC', 'NRB_N119SC_NM', 'NRB_N119SC_DSTNC']]
price_building = price_building.rename(columns = {
    'SIGNGU_CD' : '시군구코드',
    'EMDL_CD' : '읍면동리코드',
    'LNNO_ADRES' : '지번구분',
    'XCRDT' : '위도',
    'YCRDT' : '경도', 
    'NRB_GNHSP_NM' : '인접종합병원명',
    'NRB_GNHSP_ADRES' : '인접종합병원주소',
    'NRB_GNHSP_DSTNC' : '인접종합병원거리',
    'NRB_EGYT_NM' : '인접응급의료기관명',
    'NRB_EGYT_ADRES' : '인접종합병원주소',
    'NRB_EGYT_DSTNC' : '인접응급의기관거리',
    'NRB_N119SC_NM' : '인접119안접신고센터명',
    'NRB_N119SC_ADRES' : '인접종합병원주소',
    'NRB_N119SC_DSTNC' : '인접199안접신고센터거리' 
})
# 용인시에 있는 로우만 뽑아오기 
# Assuming df is your DataFrame
# price_building = price_building[price_building['지번구분'].str.contains('용인')]
#용인시 처인구로 지번이 되어있는것만 가지고오기
# price_building = price_building[price_building['지번구분'].str.contains('용인시')]
price_building



In [ ]:
## 위도 경도 바꿔주기
# Assuming clinic_infra_dfs is a dictionary
price_building = pd.DataFrame.from_dict(price_building)

price_building
# 좌표계 정의
original_crs = Proj(init='epsg:5181')  # UTM-K
target_crs = Proj(init='epsg:4326')    # WGS84 (위도, 경도)

# 좌표 변환 함수 정의
def convert_coordinates(x, y):
    lon, lat = transform(original_crs, target_crs, x, y)
    return lat, lon

# '위도'와 '경도' 열 업데이트
price_building[['위도', '경도']] = price_building.apply(lambda row: pd.Series(convert_coordinates(row['위도'], row['경도'])), axis=1)

price_building
emergency_clinic_location = price_building[['지역',	'위도',	'경도',	'인접종합병원명',	'인접종합병원주소',	'인접종합병원거리',	'인접응급의료기관명',	'인접응급의기관거리',	'인접119안접신고센터명',	'인접199안접신고센터거리']]
emergency_clinic_location


In [ ]:
emergency_clinic_location.to_csv('./pre_data/infra/emergency_clinic_location_temp.csv')


In [ ]:
### 시군구코드 처리 
"""
용인시 처인구 41461
용인시 기흥구 41463
용인시 수지구 41465
""" 
# 주어진 데이터프레임 (df)를 사용하고, 시군구코드에 따라 지역구 이름을 매핑
region_mapping = {41461: '처인구', 41463: '기흥구', 41465: '수지구'}
price_building
price_building['지역'] = price_building['시군구코드'].map(region_mapping)
price_building.drop('시군구코드', axis=1, inplace=True) #시군구 코드 제거
# price_building.to_csv('./pre_data/infra/emergency_clinic_location.csv')  

# price_building = pd.read_csv('./pre_data/infra/emergency_clinic_location.csv')

### 읍면동리(법정동) 처리
# 법정동 엑셀파일 불러오기
eupmyun_df = pd.read_excel('./data/infra/법정동_20231010.xlsx')
# '시군구명' 열에 '용인시'가 포함된 행만 선택 (NaN 값 포함)
yongin_data = eupmyun_df[eupmyun_df['시군구명'].str.contains('용인시', na=False)]

# 결과 확인
eupmyun_df = yongin_data

eupmyun_df

# '법정동코드'에서 뒤에서 5자리 추출하여 '읍면동리코드' 열 생성
eupmyun_df['읍면동리코드'] = eupmyun_df['법정동코드'].astype(str).str[-5:]

# '읍면동명' 열 추출
eupmyun_df['읍면동명'] = eupmyun_df['읍면동명']

# 결과 확인
eupmyun_df[['읍면동리코드', '읍면동명']]

# '읍면동리코드'를 문자열로 변환하여 데이터 형식을 일치시킴
eupmyun_df['읍면동리코드'] = eupmyun_df['읍면동리코드'].astype(int)

# 읍면동리코드를 기반으로 데이터프레임 합치기
merged_df = pd.merge(price_building, eupmyun_df, how='left', left_on='읍면동리코드', right_on='읍면동리코드')

merged_df['시군구명'] = merged_df['시군구명'].str.replace('용인시(.*)', r'\1', regex=True)

# 합친 결과에서 '지역' 칼럼 추가
merged_df['지역'] = merged_df['시군구명']


# 원하는 컬럼만 가지고 오기
eupmyun_df = merged_df[['지역', '읍면동명', '동리명', '위도', '경도', '인접종합병원명', '인접종합병원주소', '인접종합병원거리', '인접응급의료기관명', '인접응급의기관거리', '인접119안접신고센터명', '인접199안접신고센터거리',]]
# eupmyun_df = eupmyun_df.drop('Unnamed: 0', axis =1)
# eupmyun_df.to_csv('./pre_data/infra/emergency_clinic_location.csv')

eupmyun_df

In [ ]:
## 위도 경도 바꿔주기
# Assuming clinic_infra_dfs is a dictionary
price_building = pd.DataFrame.from_dict(price_building)

price_building
# 좌표계 정의
original_crs = Proj(init='epsg:5181')  # UTM-K
target_crs = Proj(init='epsg:4326')    # WGS84 (위도, 경도)

# 좌표 변환 함수 정의
def convert_coordinates(x, y):
    lon, lat = transform(original_crs, target_crs, x, y)
    return lat, lon

# '위도'와 '경도' 열 업데이트
price_building[['위도', '경도']] = price_building.apply(lambda row: pd.Series(convert_coordinates(row['위도'], row['경도'])), axis=1)

price_building
emergency_clinic_location = price_building[['지역',	'위도',	'경도',	'인접종합병원명',	'인접종합병원주소',	'인접종합병원거리',	'인접응급의료기관명',	'인접응급의기관거리',	'인접119안접신고센터명',	'인접199안접신고센터거리']]
emergency_clinic_location.to_csv('./pre_data/infra/emergency_clinic_location.csv')
emergency_clinic_location

#### walk_infra(도보접근가능시설수)

In [ ]:
## 데이터 로드 및 칼럼명 변경,
import pandas as pd
import numpy as np
for i in range(1,12):
    PATH = f'./data/infra/walk_infra_{i}.csv'
    print(PATH.split('/')[-1])
    walk_infra = pd.read_csv(PATH)
    print(f'row 개수 : {walk_infra.shape}')

    ### 법정동 코드 파일
    dir_df = pd.read_excel('./data/infra/법정동코드7자리.xlsx')

    # Step 1: Merge the two dataframes based on LEGALDONG_CD and 법정동 코드
    walk_infra = walk_infra.merge(dir_df, left_on='LEGALDONG_CD', right_on='법정동 코드', how='inner')

    # Drop the redundant 법정동 코드 column
    walk_infra.drop('법정동 코드', axis=1, inplace=True)

    # 필요없는 칼럼 제거 
    walk_infra = walk_infra.drop(columns=[
        'DATA_STRD_YM','PNU', 'LOT_NO', 'POI_CD', 'CTPR_CD', 'SIGNGU_CD', 'LEGALDONG_CD'
    ], axis=1)
    # '지역구'와 '읍면동' 열 추출
    geo_cols = walk_infra[['지역구', '읍면동']]

    # 나머지 열 추출
    other_cols = walk_infra.drop(columns=['지역구', '읍면동'])

    # '지역구'와 '읍면동'이 앞에 오도록 새로운 DataFrame 생성
    walk_infra = pd.concat([geo_cols, other_cols], axis=1)

    # 칼럼명 변경

    # 나머지 칼럼들을 '1'부터 '30'까지의 숫자로 순서대로 변경
    walk_infra.columns = ['지역구', '읍면동', 'WLKG_APRC_POSBL_WHOL_FCLTY_CNT'] + list(range(1, 31))

    # 변경된 DataFrame 확인
    walk_infra = walk_infra.rename(columns={'WLKG_APRC_POSBL_WHOL_FCLTY_CNT' : '도보접근가능전체시설수'})

    # 저장
    walk_infra.to_csv(os.path.join(f'./pre_data/infra/', PATH.split("/")[-1]))
    print(walk_infra.shape)
    
#모든데이터 다 쌓기
walk_infra = pd.read_csv('./pre_data/infra/walk_infra_1.csv')
for i in range(2,12):
    walk_infra_pre_data = f'./pre_data/infra/walk_infra_{i}.csv' 
    tmp = pd.read_csv(walk_infra_pre_data)
    
    walk_infra = pd.concat([walk_infra, tmp], axis = 0, ignore_index=True)
walk_infra = walk_infra.drop('Unnamed: 0', axis =1)
walk_infra.to_csv('./pre_data/infra/walk_infra.csv')
    

In [ ]:
walk_infra = pd.read_csv('./pre_data/infra/walk_infra.csv')
walk_infra = walk_infra.drop('Unnamed: 0', axis=1)
### 0~15분
walk_infra_15 = walk_infra[walk_infra.columns[range(0,18)]]
# 15분까지 합한 걸 가지고 오고
walk_infra_15['15times'] = walk_infra.loc[:, '1':'15'].sum(axis=1)
# 나머지 칼럼 제거 
walk_infra_15.drop(columns=[str(i) for i in range(1,16)], inplace = True)
walk_infra_15.to_csv('./pre_data/infra/walk_infra_15.csv')

### 0~30분
walk_infra_30 = walk_infra
# 30분까지 합한 걸 가지고 오고
walk_infra_30['30times'] = walk_infra.loc[:, '1':'30'].sum(axis=1)
# # 나머지 칼럼 제거 
walk_infra_30.drop(columns=[str(i) for i in range(1,31)], inplace = True)
# 저장
walk_infra_30.to_csv('./pre_data/infra/walk_infra_30.csv')
walk_infra_30


### population : 용인시 전체인구 및 기초수급자, 어린이, 노인 데이터

#### all : 용인시 전체인구를 지역구 읍면동별 세대수, 등록인구, 65세 이상 고립자를 알려줌

In [ ]:
all_df = population_dfs['all'][['시군구', '읍면동', '등록인구', '65세이상 고령자']] 
all_df.to_csv('./pre_data/population/all.csv')

#### basic : 용인시의 기초수급자들에 대한 지역구, 읍면동, 가구수합계, 수급권자수 합계를 보여줌

In [ ]:
population_dfs['basic'].to_csv('./pre_data/population/basic.csv')

#### child : 용인시 전체 읍에 대한 미성년 인구수를 알려줌

In [ ]:
population_dfs['child'].to_csv('./pre_data/population/child.csv')

#### company : 시군구별 연도별 건강보험가입자수를 알려줌

In [ ]:
population_dfs['company'][['시군구', '연도', '적용인구 수']]

### 데이터전처리

In [ ]:
### 데이터 병합
clinic_rate_df = pd.read_csv('./pre_data/clinic/clinic_rate.csv')
# 숫자로 변환하는 함수 정의
def convert_to_numeric(value):
    return int(value.replace(',', '').replace(' ', '').strip())

# '지급건수', '내원일수', '진료일수', '총진료비' 열의 공백과 쉼표 제거하고 숫자로 변환
clinic_rate_df[['지급건수', '내원일수', '진료일수', '총진료비', '공단부담', '본인부담']] = clinic_rate_df[['지급건수', '내원일수', '진료일수', '총진료비', '공단부담', '본인부담']].applymap(convert_to_numeric)

clinic_rate_df=clinic_rate_df.drop(columns=['Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1','Unnamed: 0'])

In [ ]:
### clinic_rate 와 건강보험료 납입 가입자 사이의 관계 -> 진료비를 많이 내는 곳에는 업무지구에 대한 관련성
wadge_df = pd.read_csv('./pre_data/clinic/wadge.csv')
wadge_df  = wadge_df.rename(columns={'시군구' : '지역'})
clinic_rate_df['건보료 건수'] = wadge_df['건수 합계']
clinic_rate_df['건보료 금액'] = wadge_df['금액 합계']
clinic_rate_df

In [ ]:
# 연도별 각 항목의 변화율 계산
clinic_rate_df['지급건수_변화율'] = clinic_rate_df.groupby('지역')['지급건수'].pct_change() * 100
clinic_rate_df['내원일수_변화율'] = clinic_rate_df.groupby('지역')['내원일수'].pct_change() * 100
clinic_rate_df['진료일수_변화율'] = clinic_rate_df.groupby('지역')['진료일수'].pct_change() * 100
clinic_rate_df['총진료비_변화율'] = clinic_rate_df.groupby('지역')['총진료비'].pct_change() * 100

# 그래프 그리기
plt.figure(figsize=(16, 12))

plt.subplot(121)
for region in clinic_rate_df['지역'].unique():
    region_data = clinic_rate_df[clinic_rate_df['지역'] == region]
    plt.scatter(region_data['연도'], region_data['지급건수'], label=region)
    # 같은 구에 속하는 데이터 포인트들을 연결하는 선 추가
    plt.plot(region_data['연도'], region_data['지급건수'], linestyle='dashed', alpha=0.5, linewidth = 19)
plt.title('연도별 지역별 지급건수 변화율', size = 15)
plt.xlabel('연도', size = 15 )
plt.ylabel('지급건수', size = 15 )
plt.legend()
plt.grid(True)

plt.subplot(122)
for region in clinic_rate_df['지역'].unique():
    region_data = clinic_rate_df[clinic_rate_df['지역'] == region]
    plt.scatter(region_data['연도'], region_data['총진료비'], label=region)
    
    # 같은 구에 속하는 데이터 포인트들을 연결하는 선 추가
    plt.plot(region_data['연도'], region_data['총진료비'], linestyle='dashed', alpha=0.5, linewidth = 19)

plt.title('연도별 지역별 총진료비 변화율', size = 15)
plt.xlabel('연도', size = 15 )
plt.ylabel('총진료비', size = 15 )
plt.legend()
plt.grid(True)
plt.suptitle("용인시 기흥구/수지구/처인구 진료비용 비교 분석", size = 30)

# plt.rc('font', size=10)        # 기본 폰트 크기
# plt.rc('axes', labelsize=20)   # x,y축 label 폰트 크기
# plt.rc('xtick', labelsize=40)  # x축 눈금 폰트 크기 
# plt.rc('ytick', labelsize=20)  # y축 눈금 폰트 크기
plt.rc('legend', fontsize=20)  # 범례 폰트 크기
# plt.rc('figure', titlesize=10) # figure title 폰트 크기
plt.tight_layout()
plt.show()
plt.savefig('./result/용인시 지역구별 연도별 진료비용 비교 분석.png')
plt.close()


In [ ]:
## 건보료 금액과 총진료비의 관계 이해
# 산점도 그리기
plt.figure(figsize=(10, 6))
sns.scatterplot(x='건보료 금액', y='총진료비', data=clinic_rate_df, hue='지역', palette={'기흥구': 'red', '수지구': 'green', '처인구': 'blue'})
plt.title('건보료금액-총진료비 상관계수 분석')
plt.xlabel('건보료 금액')
plt.ylabel('총진료비')
plt.legend()

# 상관 계수 계산
correlation_matrix = clinic_rate_df[['건보료 금액', '총진료비']].corr()
correlation_coefficient = correlation_matrix.loc['건보료 금액', '총진료비']

# 상관 계수를 플롯에 표시 (좌측 아래)
plt.text(0.7, 0.05, f'상관 계수(Corr): {correlation_coefficient:.4f}', transform=plt.gca().transAxes, fontsize=12, verticalalignment='bottom', bbox=dict(boxstyle='round', alpha=0.1))
plt.rc('font', size=10)        # 기본 폰트 크기
plt.rc('axes', labelsize=10)   # x,y축 label 폰트 크기
plt.rc('xtick', labelsize=10)  # x축 눈금 폰트 크기 
plt.rc('ytick', labelsize=10)  # y축 눈금 폰트 크기
plt.rc('legend', fontsize=10)  # 범례 폰트 크기
plt.rc('figure', titlesize=10) # figure title 폰트 크기
plt.show()
plt.savefig('./result/건보료금액-총진료비 상관계수 분석.png')
plt.close()

## clinic_rate_df의 결측치 처리 및 정규화
clinic_rate_df
# clinic_rate_df = clinic_rate_df.drop('Unnamed: 0', axis = 1)

# 변화율에 대한 결측치 처리
clinic_rate_df = clinic_rate_df.dropna()
from sklearn.preprocessing import MinMaxScaler

# 정규화할 열 선택
columns_to_normalize = ['지급건수', '내원일수', '진료일수', '총진료비', '공단부담', '본인부담', '건보료 건수', '건보료 금액']

# Min-Max Scaling 수행
scaler = MinMaxScaler()
clinic_normalized = clinic_rate_df.copy()
clinic_normalized[columns_to_normalize] = scaler.fit_transform(clinic_rate_df[columns_to_normalize])

# 정규화된 데이터프레임 출력
clinic_normalized

## 건보료 금액과 총진료비의 관계 이해
# 산점도 그리기
plt.figure(figsize=(10, 6))
sns.scatterplot(x='건보료 금액', y='총진료비', data=clinic_normalized, hue='지역', palette={'기흥구': 'red', '수지구': 'green', '처인구': 'blue'})
plt.title('건보료금액-총진료비 상관계수 분석-정규화')
plt.xlabel('건보료 금액')
plt.ylabel('총진료비')
plt.legend()

# 상관 계수 계산
correlation_matrix = clinic_normalized[['건보료 금액', '총진료비']].corr()
correlation_coefficient = correlation_matrix.loc['건보료 금액', '총진료비']

# 상관 계수를 플롯에 표시 (좌측 아래)
plt.text(0.7, 0.05, f'상관 계수(Corr): {correlation_coefficient:.4f}', transform=plt.gca().transAxes, fontsize=12, verticalalignment='bottom', bbox=dict(boxstyle='round', alpha=0.1))
plt.rc('font', size=10)        # 기본 폰트 크기
plt.rc('axes', labelsize=10)   # x,y축 label 폰트 크기
plt.rc('xtick', labelsize=10)  # x축 눈금 폰트 크기 
plt.rc('ytick', labelsize=10)  # y축 눈금 폰트 크기
plt.rc('legend', fontsize=10)  # 범례 폰트 크기
plt.rc('figure', titlesize=10) # figure title 폰트 크기
plt.show()
plt.savefig('./result/건보료금액-총진료비 상관계수 분석-정규화.png')
plt.close()
clinic_normalized.to_csv('./pre_data/clinic/clinic_rate_normal.csv')

### 의료시설(산후조리원, 병원 및 의원, 약국, 치과병원)의 위치 좌표를 이용한 밀집 지역 확인 분석

In [ ]:
### 지역에 따른 일일 환자수 계산
# 대형병원 - 의원 구분 
big_clinic_data = pd.read_excel('./pre_data/clinic_infra/big_clinic.xlsx')
# big_clinic_data = big_clinic_data.rename(columns={'지역구' : '지역', '의료인수(명)' : '의사수'})
# big_clinic_data.to_csv('./pre_data/clinic_infra/big_clinic.csv')

small_clinic_data = pd.read_excel('./pre_data/clinic_infra/small_clinic.xlsx')
# small_clinic_data = small_clinic_data.rename(columns={'지역구' : '지역', '의료인수(명)' : '의사수'})
# small_clinic_data = small_clinic_data.drop('Unnamed: 0', axis = 1)
# small_clinic_data.to_csv('./pre_data/clinic_infra/small_clinic.csv')

big_clinic_data = pd.read_csv('./pre_data/clinic_infra/big_clinic.csv')
small_clinic_data = pd.read_csv('./pre_data/clinic_infra/small_clinic.csv')

clinic_rate_data = pd.read_csv('./pre_data/clinic/clinic_rate.csv')
# clinic_rate_data = clinic_rate_data.drop('Unnamed: 0', axis = 1)
# clinic_rate_data.to_csv('./pre_data/clinic/clinic_rate.csv')
# mom_data = pd.read_csv('./pre_data/clinic_infra/mom.csv')
# # 좌표계 변환 함수 정의
# def xy_to_latlon(x, y):
#     in_proj = Proj(init='epsg:5186')  # UTM-K 좌표계
#     out_proj = Proj(init='epsg:4326')  # WGS84 좌표계

#     lon, lat = transform(in_proj, out_proj, x, y)
#     return lat, lon
# mom_data['좌표정보(X)'], mom_data['좌표정보(Y)'] = zip(*mom_data.apply(lambda row: xy_to_latlon(row['좌표정보(X)'], row['좌표정보(Y)']), axis=1))
# print(mom_data)
# mom_data = mom_data.rename(columns={'좌표정보(X)' : '위도', '좌표정보(Y)' : '경도'})
# mom_data.to_csv('./pre_data/clinic_infra/mom.csv')
mom_data = pd.read_csv('./pre_data/clinic_infra/mom.csv')


dentist_data = pd.read_csv('./pre_data/clinic_infra/dentist.csv')
# dentist_data = dentist_data.rename(columns={'소재지지번주소' : '지역', 'WGS84위도' : '위도', 'WGS84경도' : '경도', '의료인수(명)' : '의사수'})

pharmacy_data = pd.read_csv('./pre_data/clinic_infra/pharmacy.csv')
# pharmacy_data = pharmacy_data.rename(columns={'지역구': '지역','좌표(x)' : '경도', '좌표(y)' : '위도'})

emergency_clinic_data = pd.read_csv('./pre_data/infra/emergency_clinic_location.csv')

wadge_data = pd.read_csv('./pre_data/clinic/wadge.csv')
mom_data

In [ ]:
# 필요한 칼럼 추출하기
big_clinic_locations = big_clinic_data[['지역', '사업장명', '의사수', '위도', '경도']]
small_clinic_locations = small_clinic_data[['지역', '사업장명', '의사수', '위도', '경도']]
dentist_locations = dentist_data[['소재지지번주소', '사업장명', '의료인수(명)', 'WGS84위도', 'WGS84경도']]
pharmacy_locations = pharmacy_data[['지역구', '요양기관명', '좌표(x)', '좌표(y)']]
mom_locations = mom_data[['사업장명', '위도', '경도', '임산부정원수']]
# 결측치 제거
big_clinic_locations = big_clinic_locations.dropna(axis= 0)
small_clinic_locations = small_clinic_locations.dropna(axis= 0)
dentist_locations = dentist_locations.dropna(axis= 0)
pharmacy_locations = pharmacy_locations.dropna(axis = 0)
mom_locations = mom_locations.dropna(axis= 0)

In [ ]:
import folium
from folium.plugins import HeatMap

# 중심 좌표 설정: 용인시
map_center = [37.1778, 127.178]

# 지도 생성
map_osm = folium.Map(location=map_center, zoom_start=12)

# 병원 Clinic 위치 표시
for index, row in big_clinic_locations.iterrows():
    popup_content = f"병원명: {row['사업장명']}<br>노인 및 전문의사수: {row['의사수']}"
    folium.Marker([row['위도'], row['경도']], popup=popup_content, icon=folium.Icon(color='red')).add_to(map_osm)

# 의원 Clinic 위치 표시
for index, row in small_clinic_locations.iterrows():
    popup_content = f"의원명: {row['사업장명']}<br>의원 의사수: {row['의사수']}"
    folium.Marker([row['위도'], row['경도']], popup=popup_content, icon=folium.Icon(color='blue')).add_to(map_osm)

# Dentist 위치 표시
for index, row in dentist_locations.iterrows():
    popup_content = f"치과명: {row['사업장명']}<br>치과 의사수: {row['의료인수(명)']}"
    folium.Marker([row['WGS84위도'], row['WGS84경도']], popup=popup_content, icon=folium.Icon(color='green')).add_to(map_osm)

# Pharmacy 위치 표시
for index, row in pharmacy_locations.iterrows():
    popup_content = f"약국명: {row['요양기관명']}<br>"
    folium.Marker([row['좌표(y)'], row['좌표(x)']], popup=popup_content, icon=folium.Icon(color='purple')).add_to(map_osm)

# 산후조리원 위치 표시
for index, row in mom_locations.iterrows():
    popup_content = f"산후조리원 명: {row['사업장명']}<br> 임산부정원수: {row['임산부정원수']}"
    folium.Marker([row['위도'], row['경도']], popup=popup_content, icon=folium.Icon(color='orange')).add_to(map_osm)

# 지도를 HTML 파일로 저장
map_osm.save('./result/medical_facilities_map.html')

## 응급 의료서비스 취약 주거지역 정보를 이용해 folium에 이용하기

In [ ]:
## 응급 의료서비스 취약 주거지역 정보를 이용해 folium에 이용하기
emergency_clinic_data = pd.read_csv('./pre_data/infra/emergency_clinic_location.csv')

#결측치 제거
emergency_clinic_data = emergency_clinic_data.dropna()

# 정규화해주기
col_to_norm = ['인접종합병원거리', '인접응급의기관거리', '인접199안접신고센터거리']

# Min Max Scaling으로 정규화해줄꺼
def min_max_scailing(column):
    return (column - column.min()) / (column.max() - column.min())

#선택 변수에 대한 정규화
for column in col_to_norm:
    emergency_clinic_data[column + '_norm'] = min_max_scailing(emergency_clinic_data[column])

emergency_clinic_data.to_csv('./pre_data/infra/emergency_clinic_location_norm.csv')
emergency_clinic_data


In [ ]:
# 정규화된 데이터 불러오기
emergency_clinic_data = pd.read_csv('./pre_data/infra/emergency_clinic_location_temp.csv')
# emergency_clinic_data = pd.read_csv('./pre_data/infra/emergency_clinic_location_norm.csv')

# 지도 중심 좌표 설정
map_center = [emergency_clinic_data['위도'].mean(), emergency_clinic_data['경도'].mean()]

# 지도 생성
map_osm = folium.Map(location=map_center, zoom_start=12)

## 인접199안접신고센터거리를 원으로 주변 의료 부족 지역 찾기

# 정규화된 거리 정보를 시각화하여 지도에 추가
for index, row in emergency_clinic_data.iterrows():
    # folium.Marker([row['위도'], row['경도']], 
    #               popup=f"종합병원거리: {row['인접종합병원거리_norm']:.2f}<br>응급의기관거리: {row['인접응급의기관거리_norm']:.2f}<br>안접신고센터거리: {row['인접199안접신고센터거리_norm']:.2f}",
    #               icon=folium.Icon(color='red')).add_to(map_osm)
        folium.Circle(
        location= [row['위도'], row['경도']],
        radius = row['인접응급의기관거리_norm']*100,
        fill = True, 
        color = '#e8eb34',
        fill_color = 'red',
        popup = 'Circle Popup',
        tooltip = 'Circle tool tip'
    ).add_to(map_osm)

# 지도를 HTML 파일로 저장
map_osm.save('./result/emergency_clinic_map_1.html')
emergency_clinic_data

## 칼럼 상관관계 분석

### 변수 선언 및 데이터 전처리

In [ ]:
fig = pd.read_csv('./pre_data/clinic/fig.csv')
clinic_rate = pd.read_csv('./pre_data/clinic/clinic_rate.csv')
wage = pd.read_csv('./pre_data/clinic/wadge.csv')
big_clinic = pd.read_csv('./pre_data/clinic_infra/big_clinic.csv')
small_clinic = pd.read_csv('./pre_data/clinic_infra/small_clinic.csv')
dentist = pd.read_csv('./pre_data/clinic_infra/dentist.csv')
mom = pd.read_csv('./pre_data/clinic_infra/mom.csv')
pharmacy = pd.read_csv('./pre_data/clinic_infra/pharmacy.csv')
emergency_clinic = pd.read_csv('./pre_data/infra/emergency_clinic_location.csv')
walk = pd.read_csv('./pre_data/infra/walk_infra_1.csv')
population = pd.read_csv('./pre_data/population/all.csv')
basic = pd.read_csv('./pre_data/population/basic.csv')
child = pd.read_csv('./pre_data/population/child.csv')
company = pd.read_csv('./pre_data/population/company.csv')

## 필요칼럼가져오기 
## fig : 비만 환자 필요 칼럼 추출
fig = fig[['지역', '일일환자수', '월 요양급여비용']]

## clinic_rate : 용인시 구별 병원 진료비 추이 추출 
clinic_rate = clinic_rate[clinic_rate['연도']==2022] # -> 가장 최신(2022년도 기준)으로 추출
clinic_rate = clinic_rate[['지역', '지급건수', '총진료비']] 

## wage : 시군구별 건강보혐료 금액 
wage = wage[wage['연도']==2022][['지역','건수 합계', '금액 합계']] 

## big_clinic : 병원 및 요양병원 필요 칼럼 추출
big_clinic = big_clinic[['지역', '사업장명', '의사수', '위도', '경도']]
# 의사수가 0인 것들은 모두 제거하기 및 결측치 제거 
big_clinic = big_clinic[big_clinic['의사수'] != 0.].dropna()

## small_clinic : 병원 및 요양병원 필요 칼럼 추출
small_clinic = small_clinic[['지역', '사업장명', '의사수', '위도', '경도']]
# 의사수가 0인 것들은 모두 제거하기 및 결측치 제거 
small_clinic = small_clinic[small_clinic['의사수'] != 0.].dropna()

## dentist : 치과 병의원 지역, 병원명, 의사수, 위도, 경도 가져오기
dentist = dentist[['소재지지번주소', '사업장명', '의료인수(명)', 'WGS84위도', 'WGS84경도']]
# 칼럼명 바꿔주기 및 결측치제거 
dentist = dentist.dropna() #-> 폐업한애들은 의료인수가 없음
dentist = dentist.rename(columns = {'소재지지번주소' : '지역', '의료인수' : '의사수', 'WGS84d위도' : '위도', 'WGS84경도' : '경도'})

## mom : 산후조리원 위도 경도, 임산부정원수 칼럼가져오기 및 이름 변경
mom = mom[['사업장명', '위도', '경도', '임산부정원수']].rename(columns={'임산부정원수' : '환자수'}).dropna()

## phamacy : 약국 지역, 약국명 위도 경도 칼럼 이름 변경 및 가지고오기
pharmacy= pharmacy[['지역구','요양기관명','좌표(x)','좌표(y)']].rename(columns = {'좌표(x)':'위도', '좌표(y)' : '경도', '지역구':'지역', '요양기관명':'약국명'}).dropna()

## population : 용인시 시군구, 읍면동, 등록인구, 65세이상 고령자가지고오기
population = population.groupby(['시군구','읍면동']).mean().drop(columns = {'Unnamed: 0'}).rename(columns={'시군구':'지역구'}).reset_index()

## basic : 기초수급자 구별 읍면동별 수급권자수 칼럼가지고 오기 
basic = basic[['지역','읍면동','수급권자수 합계']].rename(columns= {'지역':'지역구','수급권자수 합계' : '수급권자수'}).groupby(['지역구','읍면동']).mean().reset_index()
## child : 읍면동별 미성년수 칼럼가지고 오기
child = child[['읍면동','전체 미성년 인구수']].rename(columns={'전체 미성년 인구수' : '미성년수'}).dropna()

## company : 건강보험가입자 직장인수(청년) 칼럼 가져오기 최신 2022
company = company[company['연도']==2022][['시군구', '적용인구 수']].dropna().rename(columns={'적용인구 수':'직장인수'})

## emergency_clinic : 지역	위도	경도	인접종합병원명	인접종합병원거리	인접종합병원주소
emergency_clinic = emergency_clinic.drop(columns= {'Unnamed: 0'}).dropna()[['지역','위도','경도','인접종합병원명','인접종합병원거리','인접종합병원주소']]

# 동일한 지역구와 읍면동에 대한 동일한 값들에 대한 평균값처리
walk = walk[['지역구','읍면동','도보접근가능전체시설수']]
walk = walk.groupby(['지역구','읍면동'])['도보접근가능전체시설수'].mean().reset_index().rename(columns={'도보접근가능전체시설수':'5분이내도보접근의료시설수평균'})

## Save 필요칼럼
SAVE_PATH = './column/'
fig.to_csv(os.path.join(SAVE_PATH, 'fig.csv'))
clinic_rate.to_csv(os.path.join(SAVE_PATH, 'clinic_rate.csv'))
wage.to_csv(os.path.join(SAVE_PATH, 'wage.csv'))
big_clinic.to_csv(os.path.join(SAVE_PATH, 'big_clinic.csv'))
small_clinic.to_csv(os.path.join(SAVE_PATH, 'small_clinic.csv'))
dentist.to_csv(os.path.join(SAVE_PATH, 'dentist.csv'))
mom.to_csv(os.path.join(SAVE_PATH, 'mom.csv'))
pharmacy.to_csv(os.path.join(SAVE_PATH, 'pharmacy.csv'))
population.to_csv(os.path.join(SAVE_PATH, 'population.csv'))
basic.to_csv(os.path.join(SAVE_PATH, 'basic.csv'))
child.to_csv(os.path.join(SAVE_PATH, 'child.csv'))
company.to_csv(os.path.join(SAVE_PATH, 'company.csv'))
emergency_clinic.to_csv(os.path.join(SAVE_PATH, 'emergency_clinic.csv'))
walk.to_csv(os.path.join(SAVE_PATH, 'walk.csv'))


### population 변수정리 : 지역구	읍면동	등록인구	65세이상 고령자	수급권자수	미성년수	직장인수

In [ ]:
print(f'population columns : {population.head(1)}')
print(f'basic columns : {basic.head(1)}')
print(f'child columns : {child.head(1)}')
print(f'company columns : {company.head(1)}')

#population + basic 데이터 병합 및 결측치 처리
merged_df = pd.merge(population, basic, how='left', on='읍면동')
merged_df = merged_df.drop(columns={'지역구'}).rename(columns={'시군구':'지역구'})

# # child 병합 및 결측치 처리
merged_df = pd.merge(merged_df, child, how='left', on='읍면동')

# company : 건강보험자수 병합
# merged_df = pd.merge(merged_df, company, how='left', on='읍면동')

# 직장인수=건강보험자수 비율에 따라 읍면동에 할당해주기 (이떄 등록인구를 기준으로 함)
a,b,c = 442335,380604,268261
a_tmp = merged_df[merged_df['지역구']=='기흥구']
scaling_factor = a / a_tmp['등록인구'].sum() 
a_tmp['직장인수'] =  a_tmp['등록인구'] * scaling_factor

b_tmp = merged_df[merged_df['지역구']=='수지구']
scaling_fbctor = b / b_tmp['등록인구'].sum() 
b_tmp['직장인수'] =  b_tmp['등록인구'] * scaling_factor


c_tmp = merged_df[merged_df['지역구']=='처인구']
scbling_fbctor = c / c_tmp['등록인구'].sum() 
c_tmp['직장인수'] =  c_tmp['등록인구'] * scaling_factor

# 직장인수 병합
merged_df = pd.merge(merged_df, a_tmp[['읍면동','직장인수']], how = 'left', on = '읍면동')
merged_df = pd.merge(merged_df, b_tmp[['읍면동','직장인수']], how = 'left', on = '읍면동')
merged_df = pd.merge(merged_df, c_tmp[['읍면동','직장인수']], how = 'left', on = '읍면동')

# Summing the '직장인수' columns along the rows
merged_df['직장인수'] = merged_df[['직장인수_x', '직장인수_y', '직장인수']].sum(axis=1)

# Drop the intermediate '직장인수_x', '직장인수_y' columns
merged_df = merged_df.drop(columns=['직장인수_x', '직장인수_y'])

# 저장하기
merged_df.to_csv('./column/주요값정리/population.csv')
merged_df

### infra 변수정리

#### walk merge

In [ ]:
# walk와 population merged df간 합치기
population_mer = pd.read_csv('./column/주요값정리/population.csv')
merged_df = pd.merge(population_mer, walk[['읍면동','5분이내도보접근의료시설수평균']], how = 'left', on = '읍면동')
merged_df.to_csv('./column/주요값정리/walk-population.csv')
merged_df

#### emergency_clinic merge

### clinic_infra 변수정리 : 각 지역구별로 의사수 및 병원수 구분하기

In [ ]:
big_clinic_stat = big_clinic.groupby('지역').agg({'사업장명': 'count', '의사수': 'sum'}).reset_index()
big_clinic_stat = big_clinic_stat.rename(columns = {'사업장명' : '병원 및 요양병원수', '의사수' : '병원 및 요양병원 의사수'})
big_clinic_stat

In [ ]:
small_clinic_stat = small_clinic.groupby('지역').agg({'사업장명': 'count', '의사수': 'sum'}).reset_index()
small_clinic_stat = small_clinic_stat.rename(columns={'사업장명': '의원수', '의사수': '의원 의사수'}).drop(index=0)
small_clinic_stat

In [ ]:
dentist_stat = dentist.groupby('지역').agg({'사업장명': 'count', '의료인수(명)': 'sum'}).reset_index()
dentist_stat = dentist_stat.rename(columns={'사업장명':'치과수', '의료인수(명)' : '치과 의사수'})
dentist_stat

In [ ]:
pharmacy_stat = dentist.groupby('지역').agg({'사업장명': 'count'}).reset_index().rename(columns={'사업장명' : '약국수'})
pharmacy_stat['약사 수'] = pharmacy_stat['약국수']
pharmacy_stat

In [ ]:
## 지역구 별로 병합하기
clinic_infra = pd.merge(big_clinic_stat, small_clinic_stat[['지역', '의원수', '의원 의사수']], on = '지역')
clinic_infra = pd.merge(clinic_infra, dentist_stat[['지역','치과수','치과 의사수']], on='지역')
clinic_infra = pd.merge(clinic_infra, pharmacy_stat[['지역','약국수','약사 수']], on='지역')
# 저장
clinic_infra.to_csv('./column/주요값정리/clinic_infra.csv')


In [ ]:
## 병합하기 clinic_infra, population, walk
merge_df = pd.read_csv('./column/주요값정리/walk-population.csv')
clinic_infra = pd.read_csv('./column/주요값정리/clinic_infra.csv')

# 등록인구 합 계산
total_population = merge_df.groupby('지역구')['등록인구'].sum()

# 각 행의 등록인구 비율 계산
merge_df['등록인구비율'] = merge_df.apply(lambda row: row['등록인구'] / total_population[row['지역구']], axis=1)

# NaN 값을 0으로 채우기
merge_df.fillna(0, inplace=True)

merge_df_a = merge_df[merge_df['지역구']=='기흥구']
clinic_infra_a = clinic_infra[clinic_infra['지역']=='기흥구']
# 필요한 열만 선택
hospital_data_a = clinic_infra_a[['지역', '병원 및 요양병원수', '병원 및 요양병원 의사수', '의원수', '의원 의사수', '치과수', '치과 의사수', '약국수', '약사 수']]

# 읍면동 데이터에 대해 의사수를 배당
for col in ['병원 및 요양병원수', '병원 및 요양병원 의사수', '의원수', '의원 의사수', '치과수', '치과 의사수', '약국수', '약사 수']:
    merge_df_a[col] = merge_df_a['등록인구비율'] * hospital_data_a[col].sum()
merge_df_a

merge_df_b = merge_df[merge_df['지역구']=='수지구']
clinic_infra_b = clinic_infra[clinic_infra['지역']=='수지구']
# 필요한 열만 선택
hospital_data_b = clinic_infra_b[['지역', '병원 및 요양병원수', '병원 및 요양병원 의사수', '의원수', '의원 의사수', '치과수', '치과 의사수', '약국수', '약사 수']]

# 읍면동 데이터에 대해 의사수를 배당
for col in ['병원 및 요양병원수', '병원 및 요양병원 의사수', '의원수', '의원 의사수', '치과수', '치과 의사수', '약국수', '약사 수']:
    merge_df_b[col] = merge_df_b['등록인구비율'] * hospital_data_b[col].sum()
merge_df_b

merge_df_c = merge_df[merge_df['지역구']=='처인구']
clinic_infra_c = clinic_infra[clinic_infra['지역']=='처인구']
# 필요한 열만 선택
hospital_data_c = clinic_infra_c[['지역', '병원 및 요양병원수', '병원 및 요양병원 의사수', '의원수', '의원 의사수', '치과수', '치과 의사수', '약국수', '약사 수']]

# 읍면동 데이터에 대해 의사수를 배당
for col in ['병원 및 요양병원수', '병원 및 요양병원 의사수', '의원수', '의원 의사수', '치과수', '치과 의사수', '약국수', '약사 수']:
    merge_df_c[col] = merge_df_c['등록인구비율'] * hospital_data_c[col].sum()
merge_df_c

## 모든 지역구 다합쳐서 비율 확정시킴
merge_df = pd.concat([merge_df_a, merge_df_b, merge_df_c], ignore_index=True).drop(columns = {'Unnamed: 0.1', 'Unnamed: 0'})
merge_df.to_csv('./column/주요값정리/walk-population-clinic_infra.csv')
merge_df

### clinic 데이터 병합

In [ ]:
clinic_rate = clinic_rate.rename(columns = {'지급건수' : '환자건수'})
wage = wage.rename(columns = {'건수 합계': '건강보험료 건수 합계', '금액 합계' : '건강보험료 금액 합계'})
merge_df = pd.merge(fig, clinic_rate[['지역','환자건수', '총진료비']], on = '지역')
merge_df = pd.merge(merge_df, wage[['지역', '건강보험료 건수 합계', '건강보험료 금액 합계']], on ='지역')
merge_df.to_csv('./column/주요값정리/clinic.csv')
merge_df

#### walk-population-clinic_infra.csv 데이터와 clinic data 병합

In [ ]:

merge_df

In [ ]:
## 월 요양급여비용, 환자건수, 총진료비, 건강보험료 건수 합계, 건강보험료 금액 합계에 대해서 전체 주거 데이터에 따라 비율로 가져감


## 병합하기 clinic_infra, population, walk
merge_df = pd.read_csv('./column/주요값정리/walk-population-clinic_infra.csv')
clinic_infra = pd.read_csv('./column/주요값정리/clinic.csv')
# merge_df = pd.read_csv('./column/주요값정리/walk-population.csv')
# clinic_infra = pd.read_csv('./column/주요값정리/clinic_infra.csv')

# 등록인구 합 계산
total_population = merge_df.groupby('지역구')['등록인구'].sum()

# 각 행의 등록인구 비율 계산
merge_df['등록인구비율'] = merge_df.apply(lambda row: row['등록인구'] / total_population[row['지역구']], axis=1)

# NaN 값을 0으로 채우기
merge_df.fillna(0, inplace=True)

merge_df_a = merge_df[merge_df['지역구']=='기흥구']
clinic_infra_a = clinic_infra[clinic_infra['지역']=='기흥구']
# 필요한 열만 선택
hospital_data_a = clinic_infra_a[[ '지역',	'일일환자수',	'월 요양급여비용',	'환자건수',	'총진료비',	'건강보험료 건수 합계',	'건강보험료 금액 합계']]

# 읍면동 데이터에 대해 의사수를 배당
for col in ['일일환자수',	'월 요양급여비용',	'환자건수',	'총진료비',	'건강보험료 건수 합계',	'건강보험료 금액 합계']:
    merge_df_a[col] = merge_df_a['등록인구비율'] * hospital_data_a[col].astype(float).sum()
merge_df_a

merge_df_b = merge_df[merge_df['지역구']=='수지구']
clinic_infra_b = clinic_infra[clinic_infra['지역']=='수지구']
# 필요한 열만 선택
hospital_data_b = clinic_infra_b[[ '지역',	'일일환자수',	'월 요양급여비용',	'환자건수',	'총진료비',	'건강보험료 건수 합계',	'건강보험료 금액 합계']]
hospital_data_b
# 읍면동 데이터에 대해 의사수를 배당
for col in ['일일환자수',	'월 요양급여비용',	'환자건수',	'총진료비',	'건강보험료 건수 합계',	'건강보험료 금액 합계']:
    merge_df_b[col] = merge_df_b['등록인구비율'] * hospital_data_b[col].astype(float).sum()
    
merge_df_c = merge_df[merge_df['지역구']=='처인구']
clinic_infra_c = clinic_infra[clinic_infra['지역']=='처인구']
# 필요한 열만 선택
hospital_data_c = clinic_infra_c[[ '지역',	'일일환자수',	'월 요양급여비용',	'환자건수',	'총진료비',	'건강보험료 건수 합계',	'건강보험료 금액 합계']]

# 읍면동 데이터에 대해 의사수를 배당
for col in ['일일환자수',	'월 요양급여비용',	'환자건수',	'총진료비',	'건강보험료 건수 합계',	'건강보험료 금액 합계']:
    merge_df_c[col] = merge_df_c['등록인구비율'] * hospital_data_c[col].astype(float).sum()
merge_df_c

## 모든 지역구 다합쳐서 비율 확정시킴
merge_df = pd.concat([merge_df_a, merge_df_b, merge_df_c], ignore_index=True).drop(columns = {'Unnamed: 0'})
merge_df.to_csv('./column/주요값정리/walk-population-clinic_infra-clinic.csv')
merge_df

### 모든 컬럼 상관관계분석

In [ ]:
all = pd.read_csv('./column/주요값정리/walk-population-clinic_infra-clinic.csv')
all.to_csv('./column/주요값정리/all.csv')
all.columns

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothic'

# 가정: 데이터프레임의 이름은 'all'
# 실제 변수 이름이 다르면 대체하세요.
# 'Unnamed: 0', '지역구', '읍면동' 열 제외
columns_to_exclude = ['Unnamed: 0', '지역구', '읍면동']
all_filtered = all.drop(columns=columns_to_exclude)

# 상관 행렬 계산
correlation_matrix = all_filtered.corr()

# 필요 칼럼 상관 계수 비교 히트맵 (사각형)
plt.figure(figsize=(30, 30))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('필요 칼럼 상관 계수 비교 히트맵 (사각형)', fontsize=40, fontweight='bold')  # 크기와 굵은 글씨로 변경
plt.xlabel('칼럼', fontsize=25, fontweight='bold')  # 크기와 굵은 글씨로 변경
plt.ylabel('칼럼', fontsize=25, fontweight='bold')  # 크기와 굵은 글씨로 변경
plt.xticks(rotation=45, ha='right', fontsize=20)
plt.yticks(fontsize=20)
plt.savefig('./result/corr_columns_square.png')
plt.show()
plt.close()

## 삼각형으로 플랏해서 보기
# Seaborn 스타일 설정
sns.set(style="white")
plt.rcParams['font.family'] = 'NanumGothic'

mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# 발산 색상 팔레트 생성
cmap = sns.diverging_palette(220, 20, as_cmap=True)

# 필요 칼럼 상관 계수 비교 히트맵 (삼각형)
plt.figure(figsize=(30, 30))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap=cmap, fmt='.2f', linewidths=0.5, square=True)
plt.title('필요 칼럼 상관 계수 비교 히트맵 (삼각형)', fontsize=40, fontweight='bold')  # 크기와 굵은 글씨로 변경
plt.xlabel('칼럼', fontsize=25, fontweight='bold')  # 크기와 굵은 글씨로 변경
plt.ylabel('칼럼', fontsize=25, fontweight='bold')  # 크기와 굵은 글씨로 변경
plt.xticks(rotation=45, ha='right', fontsize=20)
plt.yticks(fontsize=20)
plt.savefig('./result/corr_columns_triangle.png')
plt.show()
plt.close()


In [ ]:
import pandas as pd
import plotly.express as px

# Assuming your DataFrame is named 'all'
columns_to_exclude = ['Unnamed: 0', '지역구', '읍면동']
all_filtered = all.drop(columns=columns_to_exclude)

# Calculate the correlation matrix
correlation_matrix = all_filtered.corr()

# Create a Plotly heatmap (Square)
fig_square = px.imshow(correlation_matrix, labels=dict(color="Correlation"), x=correlation_matrix.columns, y=correlation_matrix.columns,
                       width=800, height=800, color_continuous_scale='viridis')
fig_square.update_layout(title='필요 칼럼 상관 계수 비교 히트맵 (사각형)', font=dict(family='NanumGothic', size=20))
fig_square.write_html('./result/corr_columns_square_plotly.html')

# Create a Plotly heatmap (Triangle)
fig_triangle = px.imshow(correlation_matrix.where(mask), labels=dict(color="Correlation"), x=correlation_matrix.columns, y=correlation_matrix.columns,
                         width=800, height=800, color_continuous_scale='viridis')
fig_triangle.update_layout(title='필요 칼럼 상관 계수 비교 히트맵 (삼각형)', font=dict(family='NanumGothic', size=20))
fig_triangle.write_html('./result/corr_columns_triangle_plotly.html')

# Show the Plotly figures (you can also use fig.show())

fig_square.show()
fig_triangle.show()


## 변수 중요도 분석 및 시각화

### 0. 읍면동별 중심좌표 가져오기

In [439]:
### 0. 동읍면리 중심데이터 가져오기 -> Data Source:https://herjh0405.tistory.com/156#google_vignette
df = pd.read_csv('./column/주요값정리/all.csv')
geo = pd.read_csv('./column/geo_info.csv', encoding='cp949').rename(columns={
    'center_long' : '경도',
    'center_lati' : '위도',
    'bjd_nm' : '주소명',
    'sd_nm' : '시',
    'sgg_nm' : '구',
    'emd_nm' : '읍면동',
    'li_nm' : '리',
    })
geo = geo[['주소명','시' ,'구','읍면동','리','위도','경도',]]
geo[geo['읍면동']=='풍덕천동']

,주소명,시,구,읍면동,리,위도,경도
2827,경기도 용인시 수지구 풍덕천동,경기도,용인시 수지구,풍덕천동,NaN,37.323825,127.092214


In [440]:
# 읍면동 정보와 위도 경도 정보를 조인
df = pd.merge(df, geo[['읍면동', '위도', '경도']], left_on='읍면동', right_on='읍면동', how='left')

# 중복된 컬럼 제거 및 결측치 제거
df = df.drop_duplicates(subset=['지역구','읍면동']).dropna()
# # 저장
df.to_csv('./column/주요값정리/all_geo.csv', index=False)

# 결과 출력
df.head(3)

,Unnamed: 0.1,Unnamed: 0,지역구,읍면동,등록인구,65세이상 고령자,수급권자수,미성년수,직장인수,5분이내도보접근의료시설수평균,...,약국수,약사 수,일일환자수,월 요양급여비용,환자건수,총진료비,건강보험료 건수 합계,건강보험료 금액 합계,위도,경도
0,0,0,기흥구,구갈동,32852.555556,2745.000000,785.461538,6454.0,25049.653336,10.168109,...,14.497409,14.497409,3.528080,7.668570e+05,6.208800e+05,4.240218e+07,6.208800e+05,3.156440e+07,37.277480,127.122980
1,1,1,기흥구,구성동,40265.888889,5490.555556,1054.000000,6891.0,30702.225166,8.662738,...,17.768817,17.768817,4.324208,9.399019e+05,7.609845e+05,5.197043e+07,7.609845e+05,3.868705e+07,36.371185,127.367737
7,6,6,기흥구,동백동,77224.875000,5931.875000,2.846154,0.0,58882.979268,0.000000,...,34.078340,34.078340,8.293284,1.802613e+06,1.459472e+06,9.967271e+07,1.459472e+06,7.419687e+07,37.285599,127.163197


### 1. 데이터정규화

In [442]:
## 변수 선언
df = pd.read_csv('./column/주요값정리/all.csv')
df = df.drop(columns={'Unnamed: 0.1', 'Unnamed: 0'}) #-> 시작
print(df.columns)
## 1. 정규화 시키기.
# 지역구 및 읍면동 칼럼을 제외한 모든 칼럼에 대해 정규화
df_normalized = df.drop(['지역구', '읍면동'], axis=1)
# 모든 칼럼을 정규화
df_normalized = df_normalized.div(df_normalized.sum(axis=0), axis=1)

df_norm  = pd.concat((df[['지역구', '읍면동']], df_normalized), axis= 1)
df_norm.to_csv('./column/주요값정리/all_norm.csv', index= False)
df_norm.head(3)

Index(['지역구', '읍면동', '등록인구', '65세이상 고령자', '수급권자수', '미성년수', '직장인수',
       '5분이내도보접근의료시설수평균', '등록인구비율', '병원 및 요양병원수', '병원 및 요양병원 의사수', '의원수',
       '의원 의사수', '치과수', '치과 의사수', '약국수', '약사 수', '일일환자수', '월 요양급여비용', '환자건수',
       '총진료비', '건강보험료 건수 합계', '건강보험료 금액 합계'],
      dtype='object')


,지역구,읍면동,등록인구,65세이상 고령자,수급권자수,미성년수,직장인수,5분이내도보접근의료시설수평균,등록인구비율,병원 및 요양병원수,...,치과수,치과 의사수,약국수,약사 수,일일환자수,월 요양급여비용,환자건수,총진료비,건강보험료 건수 합계,건강보험료 금액 합계
0,기흥구,구갈동,0.027994,0.021111,0.049293,0.038329,0.027994,0.039861,0.018877,0.025772,...,0.023421,0.022034,0.023421,0.023421,0.025022,0.044890,0.022551,0.022673,0.022551,0.022684
1,기흥구,구성동,0.034311,0.042227,0.066146,0.040924,0.034311,0.033960,0.023136,0.031588,...,0.028706,0.027007,0.028706,0.028706,0.030668,0.055020,0.027640,0.027790,0.027640,0.027803
2,기흥구,기흥동,0.018669,0.019568,0.033083,0.019123,0.018669,0.014118,0.012589,0.017187,...,0.015619,0.014695,0.015619,0.015619,0.016687,0.029937,0.015040,0.015121,0.015040,0.015128


### 2. 랜덤포레스트를 이용하여 총진료비에 대한 중요 변수 분석 + 총진료비에 대한 가장 높은 지역 찾기

#### -1. 총 진료비에 대한 중요도 분석 with random forest clustering

In [108]:
### 랜덤 포레스트를 이용한 변수 중요도 분석
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import plotly.express as px

# 데이터프레임 로드 (이미 로드한 경우에는 생략 가능)
df = pd.read_csv('./column/주요값정리/all_norm.csv')
df_geo = pd.read_csv('./column/주요값정리/all_geo.csv')
# 독립 변수(X)와 종속 변수(y) 설정
X = df.drop(['지역구', '읍면동'], axis=1)
y = df['총진료비']  # 예시로 '총진료비'를 종속 변수로 설정

# 랜덤 포레스트 모델 생성
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# 변수 중요도 얻기
feature_importances = model.feature_importances_

# 변수 중요도를 데이터프레임에 추가
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# 변수 중요도 기준으로 내림차순 정렬
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Plotly를 사용한 시각화
fig = px.bar(
    feature_importance_df,
    x='Importance',
    y='Feature',
    orientation='h',
    title='총 진료비에 대한 중요도 분석',
    labels={'Importance': '총 진료비 중요도', 'Feature': '변수'},
    template='plotly_dark'  # 템플릿 선택 (여러 옵션 가능)
)

# HTML 파일로 저장
fig.write_html('./result/총 진료비에 대한 중요도 분석_plotly.html')


In [109]:
## 1. 상위 변수 중심으로 입지 선정:
# 상위 5개 변수 선택 (예시로 5개 선택)
top_variables = feature_importance_df.head(5)['Feature'].tolist()
print("Top Variables:", top_variables)

## 2. 변수 간 상관 관계 고려:
# 선택한 상위 변수들 간의 상관 행렬 확인
correlation_matrix = df[top_variables].corr()
print("Correlation Matrix:\n", correlation_matrix)

## 3. 클러스터링을 활용한 지역 선정:
from sklearn.cluster import KMeans
import re

# 독립 변수 선택
X_cluster = df[top_variables]

# K-평균 클러스터링 수행 (예시로 3개의 클러스터 선택)
kmeans = KMeans(n_clusters=3, random_state=42)
df['Cluster'] = kmeans.fit_predict(X_cluster)

# 각 클러스터의 총 진료비가 높은 읍면동 선택
high_expense_clusters = df.groupby('Cluster')['총진료비'].idxmax()
selected_locations = df.loc[high_expense_clusters, ['지역구', '읍면동', '총진료비']]

# selected_locations에 지리적 정보 추가
# 읍면동에서 숫자 제거
selected_locations['읍면동'] = selected_locations['읍면동'].apply(lambda x: re.sub(r'\d+', '', str(x)))

# 읍면동을 기준으로 두 데이터프레임 합치기

selected_locations = pd.merge(selected_locations, df_geo[['지역구', '읍면동', '위도', '경도']], on=['지역구', '읍면동'], how='left')
selected_locations.dropna()

print("Selected Locations based on Clustering:")
print(selected_locations.loc[0])


## 4. 지리적 정보 고려:

import plotly.express as px

# 총 진료비가 가장 높은 지역을 큰 포인트로 강조하여 지도 시각화
fig = px.scatter_mapbox(
    selected_locations,
    lat="위도", lon="경도",
    hover_name="읍면동",
    hover_data=["총진료비"],
    color="총진료비",
    size="총진료비",
    size_max=15,  # 포인트 크기 조절
    mapbox_style="carto-positron",
    title='총 진료비가 가장 높은 지역 시각화',
)

# 타이틀 크기 조절
fig.update_layout(title_text='<b>총 진료비가 가장 높은 지역 시각화</b>', title_x=0.5, title_font_size=20)

# HTML 파일로 저장
fig.write_html('./result/총 진료비가 가장 높은 지역 시각화.html')

## 5. 모델 평가:
# 예측치 계산
predictions = kmeans.predict(X_cluster)

# 예측치를 데이터프레임에 추가
df['Predictions'] = predictions

# 예측치와 실제 총 진료비 간의 상관 관계 확인
correlation = df[['Predictions', '총진료비']].corr().iloc[0, 1]
print(f"Correlation between Predictions and Total Expense: {correlation}")

### 모든 데이터에서 가장 높은 애들만 강조하는 추가적 시각화 만들기
import plotly.express as px

# 전체 지역 시각화
fig_all = px.scatter_mapbox(
    df_geo,
    lat="위도", lon="경도",
    hover_name="읍면동",
    color="총진료비",
    size="총진료비",
    size_max=15,  # 포인트 크기 조절
    mapbox_style="carto-positron",
    title='전체 지역 시각화',
)

# 총 진료비가 가장 높은 지역을 큰 포인트로 강조
fig_selected = px.scatter_mapbox(
    selected_locations,
    lat="위도", lon="경도",
    hover_name="읍면동",
    hover_data=["총진료비"],
    color="총진료비",
    size="총진료비",
    size_max=15,  # 포인트 크기 조절
    mapbox_style="carto-positron",
    title='총 진료비가 가장 높은 지역 시각화',
)

# 전체 지도에 총 진료비가 가장 높은 지역 강조 추가
fig_all.add_trace(fig_selected.data[0])

# 타이틀 크기 조절
fig_all.update_layout(title_text='<b>전체 지역 및 총 진료비가 가장 높은 지역 시각화</b>', title_x=0.5, title_font_size=20)

# HTML 파일로 저장
fig_all.write_html('./result/전체 및 총 진료비가 가장 높은 지역 시각화.html')

Top Variables: ['건강보험료 금액 합계', '건강보험료 건수 합계', '환자건수', '총진료비', '치과 의사수']
Correlation Matrix:
              건강보험료 금액 합계  건강보험료 건수 합계      환자건수      총진료비    치과 의사수
건강보험료 금액 합계     1.000000     0.993280  0.993280  0.999989  0.946292
건강보험료 건수 합계     0.993280     1.000000  1.000000  0.993812  0.977310
환자건수            0.993280     1.000000  1.000000  0.993812  0.977310
총진료비            0.999989     0.993812  0.993812  1.000000  0.947804
치과 의사수          0.946292     0.977310  0.977310  0.947804  1.000000
Selected Locations based on Clustering:
지역구            기흥구
읍면동            신갈동
총진료비      0.025778
위도       37.283081
경도      127.101391
Name: 0, dtype: object
Correlation between Predictions and Total Expense: 0.916039708266826


In [110]:
### 전체 의료시설 다 시각화시키기 small/big clinic, dentist, pharmacy
small_clinic = pd.read_csv('./pre_data/clinic_infra/small_clinic.csv').drop(columns= {'Unnamed: 0'}).dropna()
# Remove rows with 의사수 equal to 0.0
small_clinic = small_clinic[small_clinic['의사수'] != 0]
big_clinic = pd.read_csv('./pre_data/clinic_infra/big_clinic.csv').drop(columns = {'Unnamed: 0.1', 'Unnamed: 0'}).dropna()
# Remove rows with 의사수 equal to 0.0
big_clinic = big_clinic[big_clinic['의사수'] != 0.0]
dentist = pd.read_csv('./pre_data/clinic_infra/dentist.csv').drop(columns = {'Unnamed: 0','영업상태명','폐업일자'}).dropna().rename(columns ={'소재지지번주소':'지역', '의료인수(명)' : '의사수', 'WGS84위도' :'위도', 'WGS84경도': '경도'})
# Remove rows with 의사수 equal to 0.0
dentist = dentist[dentist['의사수'] != 0.0]
pharmacy = pd.read_csv('./pre_data/clinic_infra/pharmacy.csv').drop(columns= {'Unnamed: 0', }).dropna().rename(columns={'지역구':'지역', '요양기관명' : '사업장명', '좌표(x)' : '경도', '좌표(y)' :'위도'})
pharmacy['의사수'] = 1
print(small_clinic.columns)
print(big_clinic.columns)
print(dentist.columns)
print(pharmacy.columns)

small_clinic.to_csv('./column/주요값정리/clinic_geo/small_clinic.csv', index= False)
big_clinic.to_csv('./column/주요값정리/clinic_geo/big_clinic.csv', index= False)
dentist.to_csv('./column/주요값정리/clinic_geo/dentist.csv', index= False)
pharmacy.to_csv('./column/주요값정리/clinic_geo/pharmacy.csv', index= False)

small_clinic

Index(['지역', '사업장명', '의사수', '위도', '경도'], dtype='object')
Index(['지역', '사업장명', '의사수', '위도', '경도'], dtype='object')
Index(['지역', '사업장명', '의사수', '위도', '경도'], dtype='object')
Index(['지역', '사업장명', '경도', '위도', '의사수'], dtype='object')


,지역,사업장명,의사수,위도,경도
0,수지구,진수신경외과의원,1,37.307376,127.085451
1,수지구,정인석마취통증의학과의원,2,37.340394,127.098648
2,수지구,미래연세내과의원,1,37.306474,127.085355
3,수지구,에스더산부인과의원,12,37.326738,127.109229
4,수지구,수지서울정형외과의원,1,37.324516,127.094703
...,...,...,...,...,...
737,처인구,김이비인후과의원,1,37.235227,127.209238
738,처인구,연세유플러스외과의원,1,37.234551,127.209331
739,수지구,다온가정의학과의원,1,37.336386,127.091659
740,기흥구,국민의료소비자생활협동조합기흥지점연세사랑의원,1,37.261646,127.139891


### 3. 병원 의사수에 따른 수요 조사 분석 및 시각화

In [111]:
import plotly.express as px
import plotly.graph_objects as go

# 각 데이터프레임 로드
small_clinic = pd.read_csv('./column/주요값정리/clinic_geo/small_clinic.csv')
big_clinic = pd.read_csv('./column/주요값정리/clinic_geo/big_clinic.csv')
dentist = pd.read_csv('./column/주요값정리/clinic_geo/dentist.csv')
pharmacy = pd.read_csv('./column/주요값정리/clinic_geo/pharmacy.csv')

# 공통 속성 설정
mapbox_style = "carto-positron"

# 크기를 늘리고 색상 및 마커 모양을 다르게 설정
size_max = 20
color_discrete_sequence = px.colors.qualitative.Set1

# small_clinic 시각화
fig_small_clinic = px.scatter_mapbox(
    small_clinic,
    lat="위도", lon="경도",
    hover_name="사업장명",
    color="의사수",
    size="의사수",
    size_max=size_max,
    mapbox_style=mapbox_style,
    title='의원 위치 시각화',
    color_discrete_sequence=color_discrete_sequence,
    template='plotly_dark',  # 배경색이 어두운 템플릿 사용
)

# big_clinic 시각화
fig_big_clinic = px.scatter_mapbox(
    big_clinic,
    lat="위도", lon="경도",
    hover_name="사업장명",
    color="의사수",
    size="의사수",
    size_max=size_max,
    mapbox_style=mapbox_style,
    title='병원 위치 시각화',
    color_discrete_sequence=color_discrete_sequence,
    template='plotly_dark',
)

# dentist 시각화
fig_dentist = px.scatter_mapbox(
    dentist,
    lat="위도", lon="경도",
    hover_name="사업장명",
    color="의사수",
    size="의사수",
    size_max=size_max,
    mapbox_style=mapbox_style,
    title='치과 위치 시각화',
    color_discrete_sequence=color_discrete_sequence,
    template='plotly_dark',
)

# pharmacy 시각화
fig_pharmacy = px.scatter_mapbox(
    pharmacy,
    lat="위도", lon="경도",
    hover_name="사업장명",
    color="의사수",
    size="의사수",
    size_max=size_max,
    mapbox_style=mapbox_style,
    title='약국 위치 시각화',
    color_discrete_sequence=color_discrete_sequence,
    template='plotly_dark',
)

# big_clinic 기준으로 모든 그래프를 하나의 지도로 통합
# 각 카테고리의 위치 데이터를 추가
fig_big_clinic.add_trace(go.Scattermapbox(
    lat=fig_small_clinic.data[0].lat,
    lon=fig_small_clinic.data[0].lon,
    mode='markers',
    marker=dict(
        size=fig_small_clinic.data[0].marker.size,
        color=fig_small_clinic.data[0].marker.color,
    ),
    hoverinfo='text',
    text=fig_small_clinic.data[0].hovertext,
))

fig_big_clinic.add_trace(go.Scattermapbox(
    lat=fig_dentist.data[0].lat,
    lon=fig_dentist.data[0].lon,
    mode='markers',
    marker=dict(
        size=fig_dentist.data[0].marker.size,
        color=fig_dentist.data[0].marker.color,
    ),
    hoverinfo='text',
    text=fig_dentist.data[0].hovertext,
))

fig_big_clinic.add_trace(go.Scattermapbox(
    lat=fig_pharmacy.data[0].lat,
    lon=fig_pharmacy.data[0].lon,
    mode='markers',
    marker=dict(
        size=fig_pharmacy.data[0].marker.size,
        color=fig_pharmacy.data[0].marker.color,
    ),
    hoverinfo='text',
    text=fig_pharmacy.data[0].hovertext,
))

# 중앙에 제목 타이틀 위치 및 크기 조정
fig_big_clinic.update_layout(
    title_text='<b>병원, 의원, 치과, 약국 위치 시각화</b>',
    title_x=0.5,
    title_font_size=25,
)

# HTML 파일로 저장
fig_big_clinic.write_html('./result/병원위치시각화.html')

# 시각화된 지도 표시
# fig_big_clinic.show()


#### 두개 합치기

In [114]:
import plotly.express as px

# 전체 지역 시각화
fig_all = px.scatter_mapbox(
    df_geo,
    lat="위도", lon="경도",
    hover_name="읍면동",
    color="총진료비",
    size="총진료비",
    size_max=120,  # 포인트 크기 조절
    mapbox_style="carto-positron",
    title='전체 지역 시각화',
)

# 총 진료비가 가장 높은 지역을 큰 포인트로 강조
fig_selected = px.scatter_mapbox(
    selected_locations,
    lat="위도", lon="경도",
    hover_name="읍면동",
    hover_data=["총진료비"],
    color="총진료비",
    size="총진료비",
    size_max=60,  # 포인트 크기 조절
    mapbox_style="carto-positron",
    title='총 진료비가 가장 높은 지역 시각화',
)

# 전체 지도에 총 진료비가 가장 높은 지역 강조 추가
fig_all.add_trace(fig_selected.data[0])

# 읍면동에 대한 총진료비 값 표시하는 Scatter plot 추가
fig_values = px.scatter_mapbox(
    selected_locations,
    lat="위도", lon="경도",
    text="총진료비",  # Display the '총진료비' values as text
    mapbox_style="carto-positron",
    title='총진료비 값 표시',
    color_continuous_scale="Viridis",  # Set the color scale
)

combined_hospital_df = pd.concat([small_clinic, big_clinic, dentist, pharmacy], ignore_index=True)

# 병원 위치 시각화 추가
fig_hospital_locations = px.scatter_mapbox(
    combined_hospital_df,  # Assuming you have a DataFrame containing hospital locations
    lat="위도", lon="경도",
    hover_name="사업장명",  # Display hospital names on hover
    color="의사수",
    
    size="의사수",
    size_max=80,
    mapbox_style="carto-positron",
    title='병원 위치 시각화',
)

# 전체 지도에 병원 위치 시각화 추가
fig_all.add_trace(fig_hospital_locations.data[0])

# 읍면동에 대한 총진료비 값 표시하는 Scatter plot 추가
fig_values = px.scatter_mapbox(
    selected_locations,
    lat="위도", lon="경도",
    text="총진료비",  # Display the '총진료비' values as text
    mapbox_style="carto-positron",
    title='총진료비 값 표시',
)

# 전체 지도에 총진료비 값 표시하는 Scatter plot 추가
fig_all.add_trace(fig_values.data[0])

# Add custom hover labels for all points
fig_all.update_traces(hoverinfo='text', hovertext=combined_hospital_df['사업장명'] + '<br>의사수: ' + combined_hospital_df['의사수'].astype(str))

# 타이틀 크기 조절 및 배경색 설정
fig_all.update_layout(
    title_text='<b>전체 지역, 총 진료비가 가장 높은 지역 및 병원 위치 시각화</b>',
    title_x=0.5,
    title_font_size=20,
    paper_bgcolor='white',  # Set the background color to black
    coloraxis_colorbar=dict(title='총 진료비'),  # Add colorbar title
    coloraxis_colorbar_thickness=20,  # Adjust colorbar thickness
    coloraxis_colorbar_ticklen=3,  # Adjust colorbar tick length
)
# HTML 파일로 저장
fig_all.write_html('./result/전체_총진료비_병원위치_시각화.html')



### 3. 랜덤포레스트를 이용한 의료 부족 시설 구역에 대한 중요 변수 분석 + 의료 시설이 가장 부족한 지역 찾기

In [467]:
df= pd.read_csv('./column/주요값정리/all_geo.csv')
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', '지역구', '읍면동', '등록인구', '65세이상 고령자',
       '수급권자수', '미성년수', '직장인수', '5분이내도보접근의료시설수평균', '등록인구비율', '병원 및 요양병원수',
       '병원 및 요양병원 의사수', '의원수', '의원 의사수', '치과수', '치과 의사수', '약국수', '약사 수',
       '일일환자수', '월 요양급여비용', '환자건수', '총진료비', '건강보험료 건수 합계', '건강보험료 금액 합계', '위도',
       '경도'],
      dtype='object')

## 전체 선택 칼럼을 가지고 최적입지 확인

### 1.의료 시설 부족 메트릭 계산

Index(['지역구', '읍면동', '등록인구', '65세이상 고령자', '수급권자수', '미성년수', '직장인수',
       '5분이내도보접근의료시설수평균', '등록인구비율', '병원 및 요양병원수', '병원 및 요양병원 의사수', '의원수',
       '의원 의사수', '치과수', '치과 의사수', '약국수', '약사 수', '일일환자수', '월 요양급여비용', '환자건수',
       '총진료비', '건강보험료 건수 합계', '건강보험료 금액 합계'],
      dtype='object')


,지역구,읍면동,등록인구,65세이상 고령자,수급권자수,미성년수,직장인수,5분이내도보접근의료시설수평균,등록인구비율,병원 및 요양병원수,...,치과수,치과 의사수,약국수,약사 수,일일환자수,월 요양급여비용,환자건수,총진료비,건강보험료 건수 합계,건강보험료 금액 합계
0,기흥구,구갈동,0.027994,0.021111,0.049293,0.038329,0.027994,0.039861,0.018877,0.025772,...,0.023421,0.022034,0.023421,0.023421,0.025022,0.044890,0.022551,0.022673,0.022551,0.022684
1,기흥구,구성동,0.034311,0.042227,0.066146,0.040924,0.034311,0.033960,0.023136,0.031588,...,0.028706,0.027007,0.028706,0.028706,0.030668,0.055020,0.027640,0.027790,0.027640,0.027803
2,기흥구,기흥동,0.018669,0.019568,0.033083,0.019123,0.018669,0.014118,0.012589,0.017187,...,0.015619,0.014695,0.015619,0.015619,0.016687,0.029937,0.015040,0.015121,0.015040,0.015128


In [104]:
import pandas as pd
import plotly.express as px

# 데이터프레임 로드
df = pd.read_csv('./column/주요값정리/all.csv')
# df = df.drop(columns={'Unnamed: 0.1', 'Unnamed: 0'})
df.to_csv('./column/주요값정리/all.csv', index=False)

# 의료 시설 부족 메트릭을 계산하는 함수
def calculate_healthcare_shortage_metric(row):
    healthcare_facilities = row['병원 및 요양병원수'] + row['의원수'] + row['치과수'] + row['약국수']
    return healthcare_facilities 

# 의료 시설 부족 메트릭 계산 및 데이터프레임에 추가
df['의료시설부족메트릭'] = (df.apply(calculate_healthcare_shortage_metric, axis=1)) / df['등록인구'].sum()

# 정규화 함수
def normalize(data):
    return (data - data.min()) / (data.max() - data.min())

# '의료시설부족메트릭'을 정규화
df['normalized_의료시설부족메트릭'] = 1- normalize(df['의료시설부족메트릭'])

# Plotly Express를 사용하여 정규화된 히스토그램 시각화
fig = px.histogram(df, x='읍면동', y='normalized_의료시설부족메트릭', color='읍면동', marginal='box',
                   title='정규화된 의료 시설 부족 메트릭 분포',
                   labels={'normalized_의료시설부족메트릭': '정규화된 의료 시설 부족 메트릭', '읍면동': '읍면동'},
                   template='plotly_white')

# 그래프 레이아웃 설정
fig.update_layout(
    xaxis_title='읍면동',
    yaxis_title='정규화된 의료 시설 부족 메트릭',
    showlegend=True
)

# 그래프 출력
fig.write_html('./result/읍면동별_정규화된_의료시설부족메트릭_히스토그램.html')


In [60]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_geo = pd.read_csv('./column/주요값정리/all_geo.csv')
df_geo = pd.merge(df_geo, df[['읍면동', 'normalized_의료시설부족메트릭']], on='읍면동')

# Geo Map 그리기
fig = make_subplots(
    rows=1, cols=2,  # 1행 2열의 서브플롯 생성
    column_widths=[0.7, 0.3],  # 첫 번째 서브플롯의 너비를 0.7, 두 번째 서브플롯의 너비를 0.3으로 설정
    specs=[[{'type': 'scattermapbox'}, {'type': 'bar'}]],  # 서브플롯의 유형 지정
    subplot_titles=['Geo Map', 'Bar Chart'],  # 서브플롯의 제목 지정
)

# Geo Map 추가
fig.add_trace(go.Scattermapbox(
    lat=df_geo["위도"],
    lon=df_geo["경도"],
    mode='markers',
    hovertext=df_geo["읍면동"] + '<br>의료시설부족메트릭: ' + df_geo["normalized_의료시설부족메트릭"].astype(str),
    marker=dict(
        size=df_geo["normalized_의료시설부족메트릭"] * 100,
        color=df_geo["normalized_의료시설부족메트릭"],
        colorscale='Viridis',
        colorbar=dict(title='정규화된 의료시설부족메트릭'),
    ),
), row=1, col=1)  # 1행 1열에 추가

# Bar Chart 추가
for idx, row in df_geo.iterrows():
    fig.add_trace(go.Bar(
        x=[row["읍면동"]],
        y=[row["normalized_의료시설부족메트릭"]],
        name=row["읍면동"],
        hoverinfo="text",
        marker=dict(color='rgba(0,255,255,0)'),  # 투명한 Bar를 추가하여 겹치게 함
    ), row=1, col=2)  # 1행 2열에 추가

# 레이아웃 설정
fig.update_layout(
    mapbox=dict(
        style="carto-positron",
        zoom=10,
        center=dict(lat=df_geo["위도"].mean(), lon=df_geo["경도"].mean()),
    ),
    title_text='<b>Geo Map with Bar Chart - 의료시설부족메트릭</b>',
    title_x=0.5,
    title_font_size=30,
    paper_bgcolor='white',
    yaxis=dict(title='Scattermapbox Y-axis'),
)

# HTML 파일로 저장
fig.write_html('./result/geo_map_with_bar_chart.html')

# 그래프 출력
fig.show()


다양한 시각화하기

In [70]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 데이터 불러오기
df_geo = pd.read_csv('./column/주요값정리/final.csv')

# 정렬
df_sorted = df_geo.sort_values(by='normalized_의료시설부족메트릭', ascending=False)

# 상위 3개, 하위 3개 선택
top_3_high = df_sorted.head(3)
bottom_3_low = df_sorted.tail(3)

# 레이아웃 설정
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=['Geo Map', 'Scatter Map', 'Bar Plot'],
    column_widths=[0.4, 0.4, 0.2],
    specs=[[{'type': 'scattermapbox'}, {'type': 'scattergeo'}, {'type': 'bar'}]]
)

# Geo Map 추가
fig.add_trace(go.Scattermapbox(
    lat=df_geo["위도"],
    lon=df_geo["경도"],
    text=df_geo["읍면동"],
    mode='markers',
    marker=dict(
        size=df_geo["normalized_의료시설부족메트릭"] * 100,
        color=df_geo["normalized_의료시설부족메트릭"],
        colorscale='Viridis',
        colorbar=dict(title='정규화된 의료시설부족메트릭'),
    ),
), row=1, col=1)

# Scatter Map 추가
fig.add_trace(go.Scattergeo(
    lat=df_geo["위도"],
    lon=df_geo["경도"],
    text=df_geo["읍면동"],
    mode='markers',
), row=1, col=2)

# Bar Plot 추가 (상위 3개, 하위 3개)
fig.add_trace(go.Bar(
    x=top_3_high["읍면동"],
    y=top_3_high["normalized_의료시설부족메트릭"],
    marker=dict(
        color=top_3_high["normalized_의료시설부족메트릭"],
        colorscale='Viridis',
        colorbar=dict(title='정규화된 의료시설부족메트릭'),
    ),
), row=1, col=3)

fig.add_trace(go.Bar(
    x=bottom_3_low["읍면동"],
    y=bottom_3_low["normalized_의료시설부족메트릭"],
    marker=dict(
        color=bottom_3_low["normalized_의료시설부족메트릭"],
        colorscale='Viridis',
        colorbar=dict(title='정규화된 의료시설부족메트릭'),
    ),
), row=1, col=3)

# 전체 레이아웃 설정
fig.update_mapboxes(style="carto-positron", zoom=10, center=dict(lat=df_geo["위도"].mean(), lon=df_geo["경도"].mean()))
fig.update_layout(title_text='Combined Map with 3 Layers')

# HTML 파일로 저장
fig.write_html('./result/final_map.html')

# 그래프 출력
fig.show()


### 끝내자... 제발.

In [102]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 데이터 불러오기
df_geo = pd.read_csv('./column/주요값정리/final.csv')

# 정렬
df_sorted = df_geo.sort_values(by='normalized_의료시설부족메트릭', ascending=False)

# 상위 3개, 하위 3개 선택
top_3_high = df_sorted.head(3)
bottom_3_low = df_sorted.tail(3)

# 각 병원, 의원, 치과, 약국 데이터 로드
hospital_df = pd.read_csv('./column/주요값정리/clinic_geo/big_clinic.csv')
clinic_df = pd.read_csv('./column/주요값정리/clinic_geo/small_clinic.csv')
dentist_df = pd.read_csv('./column/주요값정리/clinic_geo/dentist.csv')
pharmacy_df = pd.read_csv('./column/주요값정리/clinic_geo/pharmacy.csv')

# 병원, 의원, 치과, 약국 데이터를 하나의 데이터프레임으로 병합
merged_df = pd.concat([hospital_df, clinic_df, dentist_df, pharmacy_df])

# 레이아웃 설정
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Geo Map', 'Top 3 Bar Plot', 'Combined Clinic Map', 'Bottom 3 Bar Plot'],
    column_widths=[0.25, 0.75],
    row_heights=[0.6, 0.4],
    specs=[[{'type': 'scattermapbox'}, {'type': 'bar'}],
           [{'type': 'scattermapbox'}, {'type': 'bar'}]]
)

# Geo Map 추가
fig.add_trace(go.Scattermapbox(
    lat=df_geo["위도"],
    lon=df_geo["경도"],
    text=df_geo["읍면동"],
    mode='markers',
    marker=dict(
        size=df_geo["normalized_의료시설부족메트릭"] * 100,
        color=df_geo["normalized_의료시설부족메트릭"],
        colorscale='Viridis',
        colorbar=dict(title='정규화된 의료시설부족메트릭'),
    ),
), row=1, col=1)

# Top 3 Bar Plot 추가
fig.add_trace(go.Bar(
    x=top_3_high["읍면동"],
    y=top_3_high["normalized_의료시설부족메트릭"],
    marker=dict(
        color=top_3_high["normalized_의료시설부족메트릭"],
        colorscale='Viridis',
        colorbar=dict(title='정규화된 의료시설부족메트릭'),
    ),
), row=1, col=2)

# Combined Clinic Map 추가
fig.add_trace(go.Scattermapbox(
    lat=merged_df["위도"],
    lon=merged_df["경도"],
    text=merged_df["사업장명"],
    mode='markers',
    marker=dict(
        size=merged_df["의사수"] // 10,
        color=merged_df["의사수"],
        colorscale='Viridis',
        colorbar=dict(title='의사수'),
    ),
), row=2, col=1)

# Bottom 3 Bar Plot 추가
fig.add_trace(go.Bar(
    x=bottom_3_low["읍면동"],
    y=bottom_3_low["normalized_의료시설부족메트릭"],
    marker=dict(
        color=bottom_3_low["normalized_의료시설부족메트릭"],
        colorscale='Viridis',
        colorbar=dict(title='정규화된 의료시설부족메트릭'),
    ),
), row=2, col=2)

# 전체 레이아웃 설정
fig.update_mapboxes(style="carto-positron", zoom=10, center=dict(lat=df_geo["위도"].mean(), lon=df_geo["경도"].mean()))
fig.update_layout(title_text='Combined Map with Multiple Layers')

# HTML 파일로 저장
fig.write_html('./result/final_map_combined_clinic.html')

# 그래프 출력
fig.show()


In [93]:
## 상관관계 중요도 분석

### 랜덤 포레스트를 이용한 변수 중요도 분석
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import plotly.express as px

# 데이터프레임 로드 (이미 로드한 경우에는 생략 가능)
df = pd.read_csv('./column/주요값정리/final.csv')
df = df.rename(columns = {'normalized_의료시설부족메트릭': '의료시설부족점수'})
# 독립 변수(X)와 종속 변수(y) 설정
X = df.drop(['지역구', '읍면동'], axis=1)
y = df['의료시설부족점수']  # 예시로 '총진료비'를 종속 변수로 설정

# 랜덤 포레스트 모델 생성
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# 변수 중요도 얻기
feature_importances = model.feature_importances_

# 변수 중요도를 데이터프레임에 추가
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# # 변수 중요도 기준으로 내림차순 정렬
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# # Plotly를 사용한 시각화
fig = px.bar(
    feature_importance_df,
    x='Importance',
    y='Feature',
    orientation='h',
    title='의료약화 지역에 대한 요인 분석',
    labels={'Importance': '의료 약화 지역 에서의 요인 중요도', 'Feature': '요인'},
    template='plotly_dark'  # 템플릿 선택 (여러 옵션 가능)
)

# # HTML 파일로 저장
fig.write_html('./result/의료시설부족점수에 대한 요인 분석_plotly.html')
df_geo

,지역구,읍면동,등록인구,65세이상 고령자,수급권자수,미성년수,직장인수,5분이내도보접근의료시설수평균,등록인구비율,병원 및 요양병원수,...,약사 수,일일환자수,월 요양급여비용,환자건수,총진료비,건강보험료 건수 합계,건강보험료 금액 합계,위도,경도,normalized_의료시설부족메트릭
0,기흥구,구갈동,32852.555556,2745.000000,785.461538,6454.0,25049.653336,10.168109,0.056631,21.519591,...,14.497409,3.528080,7.668570e+05,6.208800e+05,4.240218e+07,6.208800e+05,3.156440e+07,37.277480,127.122980,0.719419
1,기흥구,구성동,40265.888889,5490.555556,1054.000000,6891.0,30702.225166,8.662738,0.069409,26.375588,...,17.768817,4.324208,9.399019e+05,7.609845e+05,5.197043e+07,7.609845e+05,3.868705e+07,36.371185,127.367737,0.631094
2,기흥구,동백동,77224.875000,5931.875000,2.846154,0.0,58882.979268,0.000000,0.133119,50.585037,...,34.078340,8.293284,1.802613e+06,1.459472e+06,9.967271e+07,1.459472e+06,7.419687e+07,37.285599,127.163197,0.190754
3,기흥구,마북동,32180.333333,4876.777778,372.461538,4791.0,24537.092491,9.316337,0.055472,21.079262,...,14.200766,3.455889,7.511658e+05,6.081757e+05,4.153456e+07,6.081757e+05,3.091853e+07,37.306688,127.127504,0.727428
4,기흥구,보라동,33866.000000,4639.000000,665.000000,6065.0,25822.391760,5.639623,0.058377,22.183433,...,14.944629,3.636915,7.905132e+05,6.400331e+05,4.371021e+07,6.400331e+05,3.253810e+07,37.255478,127.116360,0.707345
5,기흥구,보정동,36267.555556,4920.111111,246.769231,6666.0,27653.547149,12.802632,0.062517,23.756537,...,16.004404,3.894822,8.465713e+05,6.854201e+05,4.680986e+07,6.854201e+05,3.484549e+07,37.309355,127.107130,0.678732
6,기흥구,상갈동,43248.666667,4807.555556,411.923077,1687.0,32976.555063,9.406250,0.074551,28.329413,...,19.085078,4.644533,1.009527e+06,8.173560e+05,5.582025e+07,8.173560e+05,4.155287e+07,37.265509,127.110879,0.595557
7,기흥구,상하동,24928.000000,2656.666667,598.461538,3683.0,19007.281102,7.000000,0.042970,16.328726,...,11.000405,2.677052,5.818790e+05,4.711139e+05,3.217410e+07,4.711139e+05,2.395057e+07,37.261610,127.143473,0.813835
8,기흥구,신갈동,37351.000000,4086.000000,1110.230769,5706.0,28479.659677,9.434757,0.064385,24.466232,...,16.482514,4.011174,8.718614e+05,7.058961e+05,4.820824e+07,7.058961e+05,3.588646e+07,37.283081,127.101391,0.665823
9,기흥구,영덕동,44323.000000,3418.625000,0.000000,0.0,33795.720486,0.000000,0.076403,29.033140,...,19.559168,4.759907,1.034605e+06,8.376598e+05,5.720687e+07,8.376598e+05,4.258508e+07,37.273122,127.082502,0.582757


### 2. 부족한 읍면동 식별

### 3. 데이터 분석

### 4. 데이터시각화

## Visualization

In [117]:
import plotly.express as px

# 변수 중요도 결과 HTML 파일로드
variable_importance_fig = go.Figure()
variable_importance_fig.add_layout_image(
    sourfce= './result/총 진료비에 대한 중요도 분석_plotly.html'
)


# 전체 지역 시각화
fig_all = px.scatter_mapbox(
    df_geo,
    lat="위도", lon="경도",
    hover_name="읍면동",
    color="총진료비",
    size="총진료비",
    size_max=120,  # 포인트 크기 조절
    mapbox_style="carto-positron",
    title='전체 지역 시각화',
)
fig_all.add_trace(variable_importance_fig.data[0])
# 총 진료비가 가장 높은 지역을 큰 포인트로 강조
fig_selected = px.scatter_mapbox(
    selected_locations,
    lat="위도", lon="경도",
    hover_name="읍면동",
    hover_data=["총진료비"],
    color="총진료비",
    size="총진료비",
    size_max=60,  # 포인트 크기 조절
    mapbox_style="carto-positron",
    title='총 진료비가 가장 높은 지역 시각화',
)

# 전체 지도에 총 진료비가 가장 높은 지역 강조 추가
fig_all.add_trace(fig_selected.data[0])

# 읍면동에 대한 총진료비 값 표시하는 Scatter plot 추가
fig_values = px.scatter_mapbox(
    selected_locations,
    lat="위도", lon="경도",
    text="총진료비",  # Display the '총진료비' values as text
    mapbox_style="carto-positron",
    title='총진료비 값 표시',
    color_continuous_scale="Viridis",  # Set the color scale
)

combined_hospital_df = pd.concat([small_clinic, big_clinic, dentist, pharmacy], ignore_index=True)

# 병원 위치 시각화 추가
fig_hospital_locations = px.scatter_mapbox(
    combined_hospital_df,  # Assuming you have a DataFrame containing hospital locations
    lat="위도", lon="경도",
    hover_name="사업장명",  # Display hospital names on hover
    color="의사수",
    
    size="의사수",
    size_max=80,
    mapbox_style="carto-positron",
    title='병원 위치 시각화',
)

# 전체 지도에 병원 위치 시각화 추가
fig_all.add_trace(fig_hospital_locations.data[0])

# 읍면동에 대한 총진료비 값 표시하는 Scatter plot 추가
fig_values = px.scatter_mapbox(
    selected_locations,
    lat="위도", lon="경도",
    text="총진료비",  # Display the '총진료비' values as text
    mapbox_style="carto-positron",
    title='총진료비 값 표시',
)

# 전체 지도에 총진료비 값 표시하는 Scatter plot 추가
fig_all.add_trace(fig_values.data[0])

# Add custom hover labels for all points
fig_all.update_traces(hoverinfo='text', hovertext=combined_hospital_df['사업장명'] + '<br>의사수: ' + combined_hospital_df['의사수'].astype(str))

# 타이틀 크기 조절 및 배경색 설정
fig_all.update_layout(
    title_text='<b>전체 지역, 총 진료비가 가장 높은 지역 및 병원 위치 시각화</b>',
    title_x=0.5,
    title_font_size=20,
    paper_bgcolor='white',  # Set the background color to black
    coloraxis_colorbar=dict(title='총 진료비'),  # Add colorbar title
    coloraxis_colorbar_thickness=20,  # Adjust colorbar thickness
    coloraxis_colorbar_ticklen=3,  # Adjust colorbar tick length
)
# HTML 파일로 저장
fig_all.write_html('./result/전체_총진료비_병원위치_시각화.html')



IndexError: tuple index out of range